In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
import joblib
from sklearn.model_selection import KFold
import lightgbm as lgb
import time
import warnings
import os
from tqdm import tqdm
import gc # <<< IMPORT GARBAGE COLLECTOR MODULE

warnings.filterwarnings('ignore')

# --- 1. Configuration & Setup ---
CONFIG = {
    'n_folds': 20,
    'random_state': 42,
    'data_dir': 'processed_data',
}

# --- 2. Helper Functions ---
def smape(y_true, y_pred):
    y_true_actual = np.expm1(y_true)
    y_pred_actual = np.expm1(y_pred)
    numerator = np.abs(y_pred_actual - y_true_actual)
    denominator = (np.abs(y_true_actual) + np.abs(y_pred_actual)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- 3. Main Training Pipeline (CPU-Optimized) ---
def train_lightgbm_cpu(X, y):
    print("\n" + "="*50)
    print("🚀 Starting LightGBM CPU Training (Optimized for Max Cores)")
    print(f"   Folds: {CONFIG['n_folds']}")
    print("="*50)
    
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['random_state'])
    
    # LightGBM parameters for CPU
    lgb_params = {
        'objective': 'regression_l1',
        'metric': 'mae',
        'random_state': CONFIG['random_state'],
        'n_estimators': 4000,
        'learning_rate': 0.01,
        'num_leaves': 50,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
        'verbosity': -1,
        
        # --- KEY PARAMETER FOR MAX CPU USAGE ---
        'n_jobs': -1,  # This tells LightGBM to use all available CPU cores
    }

    oof_preds = np.zeros(len(y))
    oof_scores = []
    best_smape = np.inf
    model_save_path = 'best_cpu_lgbm_model.joblib'
    best_fold = -1

    # Main training loop
    for fold, (train_idx, val_idx) in enumerate(tqdm(kf.split(X, y), total=CONFIG['n_folds'], desc="CV Progress")):
        print(f"\n--- Fold {fold+1}/{CONFIG['n_folds']} ---")
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        print("  - Training LightGBM on all available CPU cores...")
        start_time = time.time()
        model = lgb.LGBMRegressor(**lgb_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[
                lgb.early_stopping(stopping_rounds=200, verbose=False),
                lgb.log_evaluation(period=500)
            ]
        )

        val_preds = model.predict(X_val)
        oof_preds[val_idx] = val_preds
        fold_smape = smape(y_val, val_preds)
        oof_scores.append(fold_smape)
        elapsed = time.time() - start_time
        print(f"    ✓ Done in {elapsed:.1f}s. Fold SMAPE: {fold_smape:.4f}%")

        # --- Checkpoint Saving Logic ---
        if fold_smape < best_smape:
            best_smape = fold_smape
            best_fold = fold + 1
            joblib.dump(model, model_save_path)
            print(f"    💾 New best model saved (SMAPE: {best_smape:.4f}%)")

        # --- AGGRESSIVE MEMORY CLEANUP TO PREVENT CRASHES ---
        # This is crucial when running many parallel processes
        del X_train, X_val, y_train, y_val, model, val_preds
        gc.collect() # Force Python to release the memory immediately

    # --- Final Evaluation ---
    print("\n" + "="*50)
    print("📊 Final Cross-Validation Results (OOF)")
    print("="*50)
    print(f"Mean SMAPE: {np.mean(oof_scores):.4f}% (Std: {np.std(oof_scores):.4f})")
    print(f"Best model from Fold {best_fold} is saved at '{model_save_path}'")

    # --- Retrain Final Model on Full Data ---
    print("\n" + "="*50)
    print("Retraining final model on 100% of data for submission...")
    print("="*50)
    final_model = lgb.LGBMRegressor(**lgb_params)
    final_model.fit(X, y) # Train on the complete dataset
    
    final_model_path = 'final_lgbm_cpu_model_submission.joblib'
    joblib.dump(final_model, final_model_path)
    print(f"\n✅ Final submission model saved as '{final_model_path}'")

# --- Main execution block ---
if __name__ == '__main__':
    def load_precomputed_data(data_dir):
        try:
            print("--- Loading final precomputed feature matrix for training ---")
            X = load_npz(os.path.join(data_dir, 'X_processed_train.npz'))
            y_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))
            print(f"   ✓ Features loaded. Shape: {X.shape}")
            print(f"   ✓ Labels loaded. Count: {len(y_df)}")
            return X, y_df['price_log'].values
        except FileNotFoundError as e:
            print(f"FATAL ERROR: Could not load precomputed files. {e}")
            return None, None

    X_final_train, y_final_train = load_precomputed_data(CONFIG['data_dir'])
    
    if X_final_train is not None:
        train_lightgbm_cpu(X_final_train, y_final_train)

--- Loading final precomputed feature matrix for training ---
   ✓ Features loaded. Shape: (75000, 5054)
   ✓ Labels loaded. Count: 75000

🚀 Starting LightGBM CPU Training (Optimized for Max Cores)
   Folds: 20


CV Progress:   0%|          | 0/20 [00:00<?, ?it/s]


--- Fold 1/20 ---
  - Training LightGBM on all available CPU cores...


CV Progress:   0%|          | 0/20 [01:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, csr_matrix
import joblib
import lightgbm as lgb
import time
import warnings
import os
from tqdm import tqdm
import scipy.sparse
import gc
from sklearn.model_selection import KFold

warnings.filterwarnings('ignore')

# --- 1. Configuration & Setup ---
CONFIG = {
    'n_folds': 20,
    'random_state': 42,
    'data_dir': 'processed_data',
    'gpu_verbose': 1,
    'batch_size': None,  # Set to int (e.g., 100_000) if you want to batch predictions
}

# --- 2. Helper Functions ---
def smape(y_true, y_pred):
    y_true_actual = np.expm1(y_true)
    y_pred_actual = np.expm1(y_pred)
    numerator = np.abs(y_pred_actual - y_true_actual)
    denominator = (np.abs(y_true_actual) + np.abs(y_pred_actual)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

def batch_predict(model, X, batch_size=100_000):
    """Predict in batches to reduce memory pressure."""
    if batch_size is None or X.shape[0] <= batch_size:
        return model.predict(X)
    preds = []
    for i in range(0, X.shape[0], batch_size):
        batch = X[i:i + batch_size]
        pred = model.predict(batch)
        preds.append(pred)
        del batch, pred
        gc.collect()
    return np.concatenate(preds)

# --- 3. GPU-Optimized Training Pipeline with Memory Cleanup ---
def train_lightgbm_gpu(X, y):
    print("\n" + "="*50)
    print("🚀 Starting LightGBM GPU Training (20 Folds + Memory Safe) 🚀")
    print(f"   Folds: {CONFIG['n_folds']}, Device: GPU")
    print("="*50)
    
    if not scipy.sparse.isspmatrix_csr(X):
        print("⚠️ Converting to CSR sparse format for GPU compatibility...")
        X = csr_matrix(X)
    
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['random_state'])

    params = {
        'objective': 'regression_l1',
        'metric': 'mae',
        'random_state': CONFIG['random_state'],
        'learning_rate': 0.01,
        'num_leaves': 50,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'verbose': CONFIG['gpu_verbose'],
    }

    oof_preds = np.zeros(len(y))
    oof_scores = []
    models = []

    for fold, (train_idx, val_idx) in enumerate(tqdm(kf.split(X, y), total=CONFIG['n_folds'], desc="Overall CV Progress")):
        print(f"\n--- Fold {fold+1}/{CONFIG['n_folds']} ---")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        if not scipy.sparse.isspmatrix_csr(X_train):
            X_train = csr_matrix(X_train)
        if not scipy.sparse.isspmatrix_csr(X_val):
            X_val = csr_matrix(X_val)

        print(f"  - Data shapes: Train={X_train.shape}, Val={X_val.shape}")
        print(f"  - Sparsity: {100 * (1 - X_train.nnz / np.prod(X_train.shape)):.2f}%")

        train_data = lgb.Dataset(X_train, label=y_train, free_raw_data=True)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data, free_raw_data=True)

        print("  - Starting GPU training...")
        start_time = time.time()

        try:
            model = lgb.train(
                params,
                train_data,
                valid_sets=[val_data],
                num_boost_round=4000,
                callbacks=[
                    lgb.early_stopping(stopping_rounds=200, verbose=True),
                    lgb.log_evaluation(period=50)
                ]
            )
        except Exception as e:
            print(f"❌ GPU Training FAILED! Error: {str(e)}")
            raise

        # Predict in batches if configured
        val_preds = batch_predict(model, X_val, batch_size=CONFIG['batch_size'])
        oof_preds[val_idx] = val_preds
        fold_smape = smape(y_val, val_preds)
        oof_scores.append(fold_smape)
        elapsed = time.time() - start_time
        print(f"    ✓ Done in {elapsed:.1f}s. Fold SMAPE: {fold_smape:.4f}%")

        models.append(model)

        # --- Aggressive memory cleanup ---
        del X_train, X_val, y_train, y_val, train_data, val_data, val_preds
        gc.collect()

    # --- Final Evaluation ---
    print("\n" + "="*50)
    print("📊 Overall Cross-Validation Results (OOF)")
    print("="*50)
    mean_smape = np.mean(oof_scores)
    std_smape = np.std(oof_scores)
    print(f"LGBM | Mean SMAPE: {mean_smape:.4f}% (Std: {std_smape:.4f})")

    overall_oof_smape = smape(y, oof_preds)
    print(f"\nFINAL | Overall OOF SMAPE: {overall_oof_smape:.4f}%")

    # --- Retrain Final Model on Full Data ---
    print("\n" + "="*50)
    print("Retraining final LightGBM model on 100% of data for submission...")
    print("="*50)

    if not scipy.sparse.isspmatrix_csr(X):
        X = csr_matrix(X)

    train_data_full = lgb.Dataset(X, label=y, free_raw_data=True)
    final_model = lgb.train(
        params,
        train_data_full,
        num_boost_round=4000,
        callbacks=[lgb.log_evaluation(period=100)]
    )

    final_model.save_model('final_lgbm_gpu_model.txt')
    print("\n✅ Final retrained model saved as 'final_lgbm_gpu_model.txt'")

    # Clean up final dataset
    del train_data_full
    gc.collect()

    return models

# --- Main execution block ---
if __name__ == '__main__':
    print("🔍 Verifying GPU setup...")
    try:
        test_data = np.random.rand(100, 10)
        test_label = np.random.rand(100)
        test_dataset = lgb.Dataset(test_data, label=test_label)
        test_params = {
            'device': 'gpu',
            'verbose': 1,
            'num_leaves': 4,
            'min_data_in_leaf': 1,
            'num_iterations': 2
        }
        lgb.train(test_params, test_dataset)
        print("  ✅ GPU test PASSED!")
        del test_data, test_label, test_dataset
        gc.collect()
    except Exception as e:
        print(f"  ❌ GPU test FAILED! Error: {str(e)}")
        exit(1)

    def load_precomputed_data(data_dir):
        try:
            print("\n--- Loading final precomputed feature matrix for training ---")
            X = load_npz(os.path.join(data_dir, 'X_processed_train.npz'))
            y_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))
            print(f"   ✓ Features loaded. Shape: {X.shape}")
            print(f"   ✓ Labels loaded. Count: {len(y_df)}")
            return X, y_df['price_log'].values
        except FileNotFoundError as e:
            print(f"FATAL ERROR: Could not load precomputed files. {e}")
            return None, None

    X_final_train, y_final_train = load_precomputed_data(CONFIG['data_dir'])

    if X_final_train is not None:
        models = train_lightgbm_gpu(X_final_train, y_final_train)
        # Optional: delete models if not needed to free memory
        del models
        gc.collect()

🔍 Verifying GPU setup...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 10
[LightGBM] [Info] Using GPU Device: Intel(R) RaptorLake-S Mobile Graphics Controller, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 10 dense feature groups (0.00 MB) transferred to GPU in 0.003704 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.494681
  ✅ GPU test PASSED!

--- Loading final precomputed feature matrix for training ---
   ✓ Features loaded. Shape: (75000, 5054)
   ✓ Labels loaded. Count: 75000

🚀 Starting LightGBM GPU Training (20 Folds + Memory Safe) 🚀
   Folds: 20, Device: GPU


Overall CV Progress:   0%|          | 0/20 [00:00<?, ?it/s]


--- Fold 1/20 ---
  - Data shapes: Train=(71250, 5054), Val=(3750, 5054)
  - Sparsity: 98.56%
  - Starting GPU training...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 755896
[LightGBM] [Info] Number of data points in the train set: 71250, number of used features: 5010
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: Intel(R) RaptorLake-S Mobile Graphics Controller, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (1.09 MB) transferred to GPU in 0.003600 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.708050
Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 0.707291
[100]	valid_0's l1: 0.662905
[150]	valid_0's l1: 0.635091
[200]	valid_0's l1: 0.616746
[250]	valid_0's l1: 0.60376

Overall CV Progress:   5%|▌         | 1/20 [15:11<4:48:42, 911.73s/it]

    ✓ Done in 911.5s. Fold SMAPE: 52.2492%

--- Fold 2/20 ---
  - Data shapes: Train=(71250, 5054), Val=(3750, 5054)
  - Sparsity: 98.56%
  - Starting GPU training...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 757010
[LightGBM] [Info] Number of data points in the train set: 71250, number of used features: 5010
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: Intel(R) RaptorLake-S Mobile Graphics Controller, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (1.09 MB) transferred to GPU in 0.002966 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.709383
Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 0.689856
[100]	valid_0's l1: 0.646956
[150]	valid_0's l1: 0.621902
[200]	valid_0

...

    ✓ Done in 805.6s. Fold SMAPE: 50.7359%

--- Fold 7/20 ---
  - Data shapes: Train=(71250, 5054), Val=(3750, 5054)
  - Sparsity: 98.56%
  - Starting GPU training...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 755857
[LightGBM] [Info] Number of data points in the train set: 71250, number of used features: 5010
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: Intel(R) RaptorLake-S Mobile Graphics Controller, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (1.09 MB) transferred to GPU in 0.006899 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.707550
Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 0.688216
[100]	valid_0's l1: 0.646654
[150]	valid_0's l1: 0.619857
[200]	valid_0

Overall CV Progress:  35%|███▌      | 7/20 [1:36:17<3:02:49, 843.78s/it]

    ✓ Done in 940.1s. Fold SMAPE: 51.3184%

--- Fold 8/20 ---
  - Data shapes: Train=(71250, 5054), Val=(3750, 5054)
  - Sparsity: 98.56%
  - Starting GPU training...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 756364
[LightGBM] [Info] Number of data points in the train set: 71250, number of used features: 5010
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: Intel(R) RaptorLake-S Mobile Graphics Controller, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (1.09 MB) transferred to GPU in 0.002047 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.708050
Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 0.679326
[100]	valid_0's l1: 0.637148
[150]	valid_0's l1: 0.610992
[200]	valid_0

Overall CV Progress:  40%|████      | 8/20 [1:52:35<2:57:17, 886.45s/it]

    ✓ Done in 977.6s. Fold SMAPE: 50.1295%

--- Fold 9/20 ---
  - Data shapes: Train=(71250, 5054), Val=(3750, 5054)
  - Sparsity: 98.56%
  - Starting GPU training...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 756344
[LightGBM] [Info] Number of data points in the train set: 71250, number of used features: 5010
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: Intel(R) RaptorLake-S Mobile Graphics Controller, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (1.09 MB) transferred to GPU in 0.002390 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.708050
Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 0.679194
[100]	valid_0's l1: 0.636769
[150]	valid_0's l1: 0.612162
[200]	valid_0

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, hstack
import joblib
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import time
import torch
import warnings
import os
from tqdm import tqdm
import gc

warnings.filterwarnings('ignore')


# --- 1. Configuration & Setup ---
CONFIG = {
    'n_folds': 20,  # Changed to 20 folds
    'random_state': 42,
    'data_dir': 'processed_data',
}
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


# --- 2. Helper Functions ---
def smape(y_true, y_pred):
    y_true_actual = np.expm1(y_true)
    y_pred_actual = np.expm1(y_pred)
    numerator = np.abs(y_pred_actual - y_true_actual)
    denominator = (np.abs(y_true_actual) + np.abs(y_pred_actual)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100


def clear_memory():
    """Clear memory after each fold"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


# --- 3. Main Training and Ensembling Pipeline ---
def train_lgbm_only(X, y):
    print("\n" + "="*50)
    print("🚀 Starting LightGBM Training (20 folds) 🚀")
    print(f"   Folds: {CONFIG['n_folds']}, Device: {DEVICE.upper()}")
    print("="*50)
    
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['random_state'])
    
    # LightGBM Hyperparameters with GPU settings
    lgb_params = {
        'objective': 'regression_l1', 
        'metric': 'mae', 
        'random_state': CONFIG['random_state'], 
        'n_estimators': 4000, 
        'learning_rate': 0.01, 
        'num_leaves': 50, 
        'subsample': 0.7, 
        'colsample_bytree': 0.7, 
        'reg_alpha': 0.1, 
        'reg_lambda': 0.1, 
        'n_jobs': -1, 
        'device': DEVICE
    }

    # Only store LightGBM model config
    model_config = {
        'lgbm': {'model': lgb.LGBMRegressor(**lgb_params), 'preds': np.zeros(len(y))}
    }
    oof_scores = {'lgbm': []}

    # Wrap the KFold loop with tqdm for a master progress bar
    for fold, (train_idx, val_idx) in enumerate(tqdm(kf.split(X, y), total=CONFIG['n_folds'], desc="Overall CV Progress")):
        print(f"\n--- Fold {fold+1}/{CONFIG['n_folds']} ---")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Train LightGBM
        print(f"  - Training LightGBM...")
        start_time = time.time()
        
        try:
            # Train LightGBM model
            model_config['lgbm']['model'] = lgb.LGBMRegressor(**lgb_params)
            model_config['lgbm']['model'].fit(
                X_train, y_train, 
                eval_set=[(X_val, y_val)], 
                callbacks=[
                    lgb.early_stopping(200, verbose=False), 
                    lgb.log_evaluation(period=500)
                ]
            )
        
        except Exception as e:
            if "CUDA Tree Learner was not enabled" in str(e):
                print("    ⚠️ LightGBM GPU failed. Falling back to CPU (this will be slow)...")
                cpu_params = lgb_params.copy()
                if 'device' in cpu_params:
                    del cpu_params['device']
                model_config['lgbm']['model'] = lgb.LGBMRegressor(**cpu_params)
                model_config['lgbm']['model'].fit(
                    X_train, y_train, 
                    eval_set=[(X_val, y_val)], 
                    callbacks=[
                        lgb.early_stopping(200, verbose=False), 
                        lgb.log_evaluation(period=500)
                    ]
                )
            else:
                print(f"    ⚠️ An error occurred training LightGBM: {e}. Skipping fold.")
                oof_scores['lgbm'].append(np.inf)  # Add a high score to indicate failure
                # Clear memory and continue to next fold
                del X_train, X_val, y_train, y_val
                clear_memory()
                continue

        # Make predictions and store results
        val_preds = model_config['lgbm']['model'].predict(X_val)
        model_config['lgbm']['preds'][val_idx] = val_preds
        fold_smape = smape(y_val, val_preds)
        oof_scores['lgbm'].append(fold_smape)
        elapsed = time.time() - start_time
        print(f"    ✓ Done in {elapsed:.1f}s. Fold SMAPE: {fold_smape:.4f}%")
        
        # Clear memory after each fold
        del X_train, X_val, y_train, y_val, val_preds
        del model_config['lgbm']['model']  # Delete the trained model to free memory
        clear_memory()

    # --- Final Evaluation ---
    print("\n" + "="*50)
    print("📊 Overall Cross-Validation Results (OOF)")
    print("="*50)
    
    scores = oof_scores['lgbm']
    if scores:
        print(f"LightGBM | Mean SMAPE: {np.mean(scores):.4f}% (Std: {np.std(scores):.4f})")
    
    # Calculate final OOF predictions using stored predictions
    ensemble_smape = smape(y, model_config['lgbm']['preds'])
    print(f"\n{'LIGHTGBM':<8} | Final OOF SMAPE: {ensemble_smape:.4f}%")
    
    # --- Retrain Final Model ---
    print("\n" + "="*50)
    print("Retraining final LightGBM model on 100% of data for submission...")
    print("="*50)
    
    try:
        final_model = lgb.LGBMRegressor(**lgb_params)
        final_model.fit(X, y)
    except Exception as e:
        if "CUDA Tree Learner was not enabled" in str(e):
            print("    ⚠️ GPU failed for final model. Retraining on CPU.")
            cpu_params = lgb_params.copy()
            if 'device' in cpu_params:
                del cpu_params['device']
            final_model = lgb.LGBMRegressor(**cpu_params)
            final_model.fit(X, y)
        else: 
            raise e
             
    joblib.dump(final_model, 'final_lightgbm_model_for_submission.joblib')
    print("\n✅ Final retrained model saved as 'final_lightgbm_model_for_submission.joblib'")


# --- Main execution block ---
if __name__ == '__main__':
    # This assumes your full feature matrix has been saved by a master preprocessing script
    def load_precomputed_data(data_dir):
        try:
            print("--- Loading final precomputed feature matrix for training ---")
            X = load_npz(os.path.join(data_dir, 'X_final_train.npz'))
            y_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))
            print(f"   ✓ Features loaded. Shape: {X.shape}")
            print(f"   ✓ Labels loaded. Count: {len(y_df)}")
            return X, y_df['price_log'].values
        except FileNotFoundError as e:
            print(f"FATAL ERROR: Could not load precomputed files. {e}")
            print("Please run the final, combined preprocessing script first to generate 'X_final_train.npz' and 'y_train.csv'.")
            return None, None

    X_final_train, y_final_train = load_precomputed_data(CONFIG['data_dir'])
    
    if X_final_train is not None:
        train_lgbm_only(X_final_train, y_final_train)

c:\Users\Akshit\anaconda3\envs\tf\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


--- Loading final precomputed feature matrix for training ---
   ✓ Features loaded. Shape: (75000, 9370)
   ✓ Labels loaded. Count: 75000

🚀 Starting LightGBM Training (20 folds) 🚀
   Folds: 20, Device: CUDA


Overall CV Progress:   0%|          | 0/20 [00:00<?, ?it/s]


--- Fold 1/20 ---
  - Training LightGBM...
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.


Overall CV Progress:   0%|          | 0/20 [00:17<?, ?it/s]


KeyboardInterrupt: 

In [1]:
!pip install optuna

In [ ]:
import pandas as pd
import re
from urllib.parse import urlparse

def clean_image_url(url):
    """
    Normalize and fix image URLs for Amazon data.
    """
    url = str(url).strip()
    url = url.replace(']', '').replace('(', '').replace(')', '').replace('[', '')
    # Remove any surrounding bad characters
    url = url.strip(' ;,\'"')
    # Fix schemes and leading http(s)
    if url.startswith('https://'):
        return url
    if url.startswith('http://'):
        return 'https://' + url[7:]
    # Some Amazon links might miss scheme
    if url.startswith('m.media-amazon') or url.startswith('images-amazon'):
        return 'https://' + url
    # Catch malformed or custom patterns
    if 'media-amazon' in url and not url.startswith('http'):
        return 'https://' + url
    # Final check: if still doesn't look valid, return None
    if 'media-amazon' not in url:
        return None
    return url

def clean_catalog_content(text):
    """
    Remove HTML tags, unicode artifacts, and special characters.
    Prefer only meaningful text for modeling.
    """
    if pd.isna(text):
        return ""
    # Remove HTML-like tags
    text = re.sub(r'<.*?>', ' ', str(text))
    # Remove non-standard unicode and extra whitespace
    text = re.sub(r'[\n\r\t]', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s.,:-]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_numeric_text_features(text):
    """
    Extracts key numeric or structured features from the catalog text.
    Extend these patterns as you find more types in your dataset!
    """
    features = {}
    text_l = str(text).lower()
    # Item Pack Quantity
    ipq = 1
    m_ipq = re.search(r'(item pack quantity|count|pack of|pk of|ct)\s*[:\-]?\s*(\d+)', text_l)
    if m_ipq:
        try:
            ipq = int(m_ipq.group(2))
        except:
            ipq = 1
    features['item_pack_quantity'] = ipq

    # Extract value and unit (e.g. "value 16.0 unit ounce")
    m_valunit = re.search(r'value\s*([\d.]+)\s*unit\s*([a-zA-Z]+)', text_l)
    if m_valunit:
        value, unit = float(m_valunit.group(1)), m_valunit.group(2)
    else:
        value, unit = 0.0, ""
    features['value'] = value
    features['value_unit'] = unit

    # Weight-related extraction (oz, lb, g, kg)
    weight_patterns = [
        (r'([\d.]+)\s?oz', 'oz'),
        (r'([\d.]+)\s?ounce', 'oz'),
        (r'([\d.]+)\s?lb', 'lb'),
        (r'([\d.]+)\s?pound', 'lb'),
        (r'([\d.]+)\s?kg', 'kg'),
        (r'([\d.]+)\s?g\b', 'g')
    ]
    weight_val = 0.0
    for pat, u in weight_patterns:
        m_weight = re.search(pat, text_l)
        if m_weight:
            weight_val = float(m_weight.group(1))
            features[f'weight_{u}'] = weight_val

    # Volume-related extraction (ml, l, fl oz)
    volume_patterns = [
        (r'([\d.]+)\s?ml', 'ml'),
        (r'([\d.]+)\s?l\b', 'l'),
        (r'([\d.]+)\s?fl oz', 'floz'),
        (r'([\d.]+)\s?gallon', 'gallon')
    ]
    for pat, u in volume_patterns:
        m_vol = re.search(pat, text_l)
        if m_vol:
            features[f'volume_{u}'] = float(m_vol.group(1))

    # Textual stats
    features['char_length'] = len(text)
    features['word_count'] = len(text.split())
    features['has_organic'] = int('organic' in text_l)
    features['has_gourmet'] = int('gourmet' in text_l)
    features['has_natural'] = int('natural' in text_l)
    features['has_premium'] = int('premium' in text_l)
    features['has_luxury'] = int('luxury' in text_l)
    features['has_vegan'] = int('vegan' in text_l)
    features['has_glutenfree'] = int('gluten free' in text_l or 'gluten-free' in text_l)

    return features

def preprocess_main(file_path, save_path=None):
    """
    Full robust preprocessing: load, clean, extract features, and save.
    """
    df = pd.read_csv(file_path)
    df = df.rename(columns=lambda x: x.strip().lower().replace(' ', '_'))

    df['catalog_content'] = df['catalog_content'].fillna('')
    df['catalog_content_clean'] = df['catalog_content'].apply(clean_catalog_content)
    df['image_link_clean'] = df['image_link'].apply(clean_image_url)

    # Numeric/structured text features:
    struct_feats_list = df['catalog_content'].apply(extract_numeric_text_features)
    struct_feats_df = pd.DataFrame(struct_feats_list.tolist())
    out_df = pd.concat([df, struct_feats_df], axis=1)

    # Remove samples without valid image url
    out_df = out_df[out_df['imagelink_clean'].str.startswith('https://', na=False)]

    # Remove duplicate sampleid if any
    out_df = out_df.drop_duplicates(subset=['sampleid'])

    print("Preprocessing complete. Head of dataframe:")
    print(out_df.head())

    if save_path:
        out_df.to_csv(save_path, index=False)
        print(f"Cleaned/preprocessed data saved to {save_path}")

    return out_df

# Usage sample (Update the path as needed)
# if __name__ == '__main__':
#     clean_df = preprocess_main('paste.txt', save_path='cleaned_data.csv')


In [ ]:
import os
import pandas as pd

DATASET_FOLDER = '68e8d1d70b66d_student_resource\student_resource\dataset'

# from my_preprocessing_script import preprocess_main  # Put the above script in `my_preprocessing_script.py`

train_path = os.path.join(DATASET_FOLDER, 'train.csv')
test_path = os.path.join(DATASET_FOLDER, 'test.csv')
sample_test_path = os.path.join(DATASET_FOLDER, 'sample_test.csv')
sample_test_out_path = os.path.join(DATASET_FOLDER, 'sample_test_out.csv')

# Run and save outputs for each split
train_clean = preprocess_main(train_path, save_path=os.path.join(DATASET_FOLDER, 'train_clean.csv'))
test_clean = preprocess_main(test_path, save_path=os.path.join(DATASET_FOLDER, 'test_clean.csv'))
sample_test_clean = preprocess_main(sample_test_path, save_path=os.path.join(DATASET_FOLDER, 'sample_test_clean.csv'))
sample_test_out_clean = preprocess_main(sample_test_out_path, save_path=os.path.join(DATASET_FOLDER, 'sample_test_out_clean.csv'))


In [ ]:
#!/usr/bin/env python3
"""
Amazon ML Challenge 2025 - Smart Product Pricing
Data Preprocessing Pipeline

This module implements a robust, error-handled preprocessing pipeline for the
catalog_content field, designed to handle the inconsistent and semi-structured
format found in the dataset while strictly adhering to competition constraints.

Key features:
- Comprehensive error handling with fallback mechanisms
- Unit standardization across multiple measurement systems
- Brand recognition with contextual analysis
- Premium attribute detection with semantic understanding
- Strict validation to prevent data leakage
- Detailed logging for debugging and audit purposes
- No external price lookup (compliance with competition rules)
"""

import os
import re
import logging
import warnings
import unicodedata
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional, Union, Any, Callable
from collections import defaultdict
from functools import wraps
from contextlib import contextmanager
import traceback
import json
from enum import Enum
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import string

# ======================
# CONFIGURATION & SETUP
# ======================

# Configure logging with detailed formatting
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger("amazon_ml_preprocessor")

# Suppress benign warnings but keep critical ones
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

# Competition constraints validation
MAX_MODEL_PARAMS = 8_000_000_000  # 8 billion parameters
ALLOWED_LICENSES = ["MIT", "Apache 2.0"]
EXTERNAL_PRICE_LOOKUP_PROHIBITED = True

# Unit conversion constants
UNIT_CONVERSIONS = {
    # Weight units to grams
    'oz': 28.3495, 'ounce': 28.3495, 'ounces': 28.3495,
    'lb': 453.592, 'pound': 453.592, 'pounds': 453.592,
    'kg': 1000.0, 'kilogram': 1000.0, 'kilograms': 1000.0,
    'g': 1.0, 'gram': 1.0, 'grams': 1.0,
    
    # Volume units to milliliters
    'fl_oz': 29.5735, 'fluid_ounce': 29.5735, 'fluid_ounces': 29.5735,
    'ml': 1.0, 'milliliter': 1.0, 'milliliters': 1.0,
    'l': 1000.0, 'liter': 1000.0, 'liters': 1000.0,
    'cup': 236.588, 'cups': 236.588,
    'pt': 473.176, 'pint': 473.176, 'pints': 473.176,
    
    # Count units (no conversion needed)
    'count': 1.0, 'ct': 1.0, 'item': 1.0, 'items': 1.0, 'piece': 1.0, 'pieces': 1.0
}

# Standardized unit categories
UNIT_CATEGORIES = {
    'weight': ['g', 'oz', 'lb', 'kg'],
    'volume': ['ml', 'fl_oz', 'l', 'cup', 'pt'],
    'count': ['count', 'item']
}

# Common brand list with priority scores (higher = more premium)
BRAND_PREMIUM_SCORES = {
    'Campbell\'s': 0.8, 'Kellogg\'s': 0.7, 'Coca-Cola': 0.6, 'McCormick': 0.75,
    'L\'Oréal': 1.2, 'Starbucks': 1.1, 'Hampton Farms': 0.9, 'Schweppes': 0.7,
    'Del Monte': 0.65, 'Frosted Flakes': 0.6, 'Near East': 0.55, 'Taffy Town': 0.6,
    'YumEarth': 0.7, 'PAM': 0.5, 'LÄRABAR': 0.85, 'Fisher Nuts': 0.75,
    'Michaels Of Bro': 0.8, 'SWEDISH FISH': 0.6, 'Near East': 0.55, 'Jack Link\'s': 0.7,
    'Foods Alive': 0.65, 'R.W. Knudsen': 0.6, 'NPG': 0.5, 'Happy Birthday Cookies': 0.9
}

# Premium attribute keywords with impact scores
PREMIUM_KEYWORDS = {
    # Premium quality indicators
    'organic': 0.8, 'premium': 1.0, 'gourmet': 1.2, 'artisan': 1.1, 
    'handcrafted': 1.0, 'luxury': 1.3, 'signature': 0.9, 'special reserve': 1.1,
    'premium quality': 1.0, 'craft': 0.7, 'small batch': 0.9, 'limited edition': 1.0,
    
    # Health attributes
    'gluten free': 0.6, 'non-gmo': 0.5, 'vegan': 0.7, 'keto': 0.6, 
    'dairy free': 0.5, 'sugar free': 0.4, 'kosher': 0.3, 'halal': 0.3,
    'non dairy': 0.4, 'plant based': 0.6, 'all natural': 0.5, 'natural': 0.3,
    
    # Convenience features
    'convenient': 0.3, 'easy to prepare': 0.4, 'ready in minutes': 0.5,
    'microwave ready': 0.6, 'no refrigeration': 0.4, 'lunchbox ready': 0.5,
    
    # Sustainability
    'sustainable': 0.5, 'eco friendly': 0.6, 'biodegradable': 0.4, 'recyclable': 0.3
}

# Product categories with typical price ranges (for imputation)
PRODUCT_CATEGORIES = {
    'beverages': {'median_price': 3.5, 'std_dev': 2.0, 'unit': 'fl_oz', 'std_unit': 'ml'},
    'snacks': {'median_price': 2.0, 'std_dev': 1.5, 'unit': 'oz', 'std_unit': 'g'},
    'cereal': {'median_price': 4.0, 'std_dev': 2.5, 'unit': 'oz', 'std_unit': 'g'},
    'condiments': {'median_price': 3.0, 'std_dev': 1.8, 'unit': 'oz', 'std_unit': 'g'},
    'baked_goods': {'median_price': 5.0, 'std_dev': 3.0, 'unit': 'count', 'std_unit': 'item'},
    'personal_care': {'median_price': 8.0, 'std_dev': 4.0, 'unit': 'oz', 'std_unit': 'ml'},
    'kitchenware': {'median_price': 15.0, 'std_dev': 10.0, 'unit': 'count', 'std_unit': 'item'},
    'other': {'median_price': 4.5, 'std_dev': 3.0, 'unit': 'count', 'std_unit': 'item'}
}

# ======================
# DECORATORS & UTILITIES
# ======================

def time_execution(func: Callable) -> Callable:
    """Decorator to measure and log function execution time."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        logger.debug(f"{func.__name__} executed in {end_time - start_time:.4f} seconds")
        return result
    return wrapper

def validate_input_data(func: Callable) -> Callable:
    """Decorator to validate input data structure before processing."""
    @wraps(func)
    def wrapper(df: pd.DataFrame, *args, **kwargs) -> Any:
        required_columns = ['sample_id', 'catalog_content']
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            raise ValueError(f"Missing required columns: {missing}. Required: {required_columns}")
        
        # Check for empty dataframe
        if df.empty:
            raise ValueError("Input dataframe is empty")
            
        # Check for duplicate sample_ids
        if df['sample_id'].duplicated().any():
            logger.warning("Duplicate sample_id values detected. Keeping first occurrence.")
            df = df.drop_duplicates(subset=['sample_id'], keep='first')
            
        return func(df, *args, **kwargs)
    return wrapper

def robust_retry(max_attempts: int = 3, delay: float = 0.5) -> Callable:
    """
    Decorator for retrying operations that might fail due to transient issues.
    
    Args:
        max_attempts: Maximum number of retry attempts
        delay: Delay between attempts in seconds
    """
    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    logger.warning(f"Attempt {attempt+1}/{max_attempts} failed for {func.__name__}: {str(e)}")
                    if attempt < max_attempts - 1:
                        time.sleep(delay)
            
            # If we get here, all attempts failed
            logger.error(f"All {max_attempts} attempts failed for {func.__name__}")
            raise RuntimeError(f"Operation failed after {max_attempts} attempts") from last_exception
        return wrapper
    return decorator

@contextmanager
def temporary_silence(category: Warning = UserWarning):
    """Context manager to temporarily silence specific warnings."""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category)
        yield

def sanitize_string(text: str) -> str:
    """Clean and standardize string inputs."""
    if not isinstance(text, str):
        return ""
    
    # Normalize unicode characters
    text = unicodedata.normalize('NFKC', text)
    
    # Replace various dash types with standard hyphen
    text = re.sub(r'[\u2013\u2014\u2212]', '-', text)
    
    # Standardize quotes
    text = re.sub(r'[\u2018\u2019\u201A\u201B\u2032]', "'", text)
    text = re.sub(r'[\u201C\u201D\u201E\u201F\u2033]', '"', text)
    
    # Remove control characters
    text = ''.join(c for c in text if c.isprintable() or c.isspace())
    
    # Standardize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# ======================
# CORE PREPROCESSING
# ======================

class DataValidationError(Exception):
    """Custom exception for data validation failures."""
    pass

class UnitConversionError(Exception):
    """Custom exception for unit conversion issues."""
    pass

class PreprocessingStep(Enum):
    """Enumeration of preprocessing steps for tracking and debugging."""
    PARSING = "parsing"
    UNIT_STANDARDIZATION = "unit_standardization"
    BRAND_EXTRACTION = "brand_extraction"
    PREMIUM_ATTRIBUTES = "premium_attributes"
    CATEGORY_ASSIGNMENT = "category_assignment"
    MISSING_VALUE_HANDLING = "missing_value_handling"
    FEATURE_ENGINEERING = "feature_engineering"

class PreprocessingContext:
    """Context object to track preprocessing state and metadata."""
    
    def __init__(self, is_training: bool = True):
        self.is_training = is_training
        self.steps_completed: List[PreprocessingStep] = []
        self.warnings: List[Dict[str, Any]] = []
        self.errors: List[Dict[str, Any]] = []
        self.stats: Dict[str, Any] = {
            'total_records': 0,
            'missing_value_count': 0,
            'unit_conversion_failures': 0,
            'brand_detection_count': defaultdict(int),
            'category_distribution': defaultdict(int)
        }
        self.start_time = time.time()
        self.validation_failures = 0
        self.category_data = {}  # Will store category statistics
    
    def add_warning(self, step: PreprocessingStep, message: str, sample_id: Optional[int] = None):
        """Add a warning to the context."""
        self.warnings.append({
            'step': step.value,
            'message': message,
            'sample_id': sample_id,
            'timestamp': time.time()
        })
        logger.warning(f"[{step.value}] {message}" + (f" (sample_id={sample_id})" if sample_id else ""))
    
    def add_error(self, step: PreprocessingStep, error: Exception, sample_id: Optional[int] = None):
        """Add an error to the context."""
        self.errors.append({
            'step': step.value,
            'error_type': type(error).__name__,
            'message': str(error),
            'traceback': traceback.format_exc(),
            'sample_id': sample_id,
            'timestamp': time.time()
        })
        logger.error(f"[{step.value}] {type(error).__name__}: {str(error)}" + 
                    (f" (sample_id={sample_id})" if sample_id else ""))
    
    def record_stat(self, stat_key: str, value: Any = 1):
        """Record a statistic or increment a counter."""
        if isinstance(self.stats[stat_key], dict) and not isinstance(value, dict):
            self.stats[stat_key][value] += 1
        elif stat_key in self.stats:
            if isinstance(self.stats[stat_key], (int, float)):
                self.stats[stat_key] += value
            else:
                self.stats[stat_key] = value
        else:
            self.stats[stat_key] = value
    
    def get_summary(self) -> Dict[str, Any]:
        """Get a summary of the preprocessing context."""
        return {
            'total_time': time.time() - self.start_time,
            'steps_completed': [step.value for step in self.steps_completed],
            'warning_count': len(self.warnings),
            'error_count': len(self.errors),
            'stats': dict(self.stats)
        }

def create_preprocessing_context(df: pd.DataFrame, is_training: bool = True) -> PreprocessingContext:
    """Create and initialize a preprocessing context."""
    context = PreprocessingContext(is_training)
    context.stats['total_records'] = len(df)
    return context

def validate_competition_constraints() -> None:
    """
    Validate that our approach complies with competition constraints.
    This is critical to avoid disqualification.
    """
    # Check for prohibited external price lookup
    if EXTERNAL_PRICE_LOOKUP_PROHIBITED:
        logger.info("Confirmed: No external price lookup is being performed (compliance check)")
    
    # This would be checked during model submission, but good to be aware
    logger.info(f"Model parameter limit: {MAX_MODEL_PARAMS/1e9} billion (must be <= 8B)")
    logger.info(f"Allowed licenses: {', '.join(ALLOWED_LICENSES)}")

# ======================
# PARSING FUNCTIONS
# ======================

@time_execution
def parse_catalog_content(text: str, context: PreprocessingContext, 
                          sample_id: Optional[int] = None) -> Dict[str, Any]:
    """
    Parse the semi-structured catalog_content field into structured components.
    
    Handles various inconsistent formatting patterns found in the dataset.
    
    Args:
        text: Raw catalog_content string
        context: Preprocessing context for tracking issues
        sample_id: Optional sample ID for error tracking
        
    Returns:
        Dictionary with parsed components
    """
    if not isinstance(text, str) or text.strip() == "":
        context.add_warning(PreprocessingStep.PARSING, "Empty or invalid catalog_content", sample_id)
        return {
            'item_name': '',
            'bullet_points': [],
            'product_description': '',
            'value': None,
            'unit': None,
            'raw_text': text
        }
    
    try:
        # Sanitize input text
        text = sanitize_string(text)
        
        # Initialize result structure
        result = {
            'item_name': '',
            'bullet_points': [],
            'product_description': '',
            'value': None,
            'unit': None,
            'raw_text': text
        }
        
        # Extract item name (most reliable pattern)
        item_name_match = re.search(r'Item Name:\s*(.*?)(?:Bullet Point|$|Product Description|Value:)', text)
        if item_name_match:
            result['item_name'] = item_name_match.group(1).strip()
            # Remove item name from text to avoid interference with other patterns
            text = text.replace(item_name_match.group(0), '', 1)
        else:
            # Fallback: Try to find anything before the first structured element
            first_structured = re.search(r'(Bullet Point|Product Description|Value:)', text)
            if first_structured:
                result['item_name'] = text[:first_structured.start()].strip()
                text = text[first_structured.start():]
            else:
                # Last resort: take first 50 characters as item name
                context.add_warning(PreprocessingStep.PARSING, 
                                   "Could not properly identify item name, using fallback", 
                                   sample_id)
                result['item_name'] = text[:50].strip()
        
        # Extract bullet points with multiple pattern handling
        bullet_points = []
        
        # Pattern 1: "Bullet Point X: content"
        bullet_pattern1 = r'Bullet Point\s*\d+\s*:\s*(.*?)(?=(?:Bullet Point\s*\d+\s*:|Product Description|Value:|$))'
        matches1 = re.findall(bullet_pattern1, text, re.DOTALL)
        bullet_points.extend([m.strip() for m in matches1 if m.strip()])
        
        # Pattern 2: "• content" or "- content"
        bullet_pattern2 = r'(?:•|-|●|·)\s*(.*?)(?=(?:•|-|●|·|Product Description|Value:|$))'
        matches2 = re.findall(bullet_pattern2, text, re.DOTALL)
        bullet_points.extend([m.strip() for m in matches2 if m.strip()])
        
        # Pattern 3: Numbered list "1. content", "2. content", etc.
        bullet_pattern3 = r'\d+\.\s*(.*?)(?=\d+\.\s*|$)'
        matches3 = re.findall(bullet_pattern3, text, re.DOTALL)
        bullet_points.extend([m.strip() for m in matches3 if m.strip()])
        
        # Remove duplicates while preserving order
        seen = set()
        result['bullet_points'] = [x for x in bullet_points 
                                  if not (x in seen or seen.add(x)) and x]
        
        # Extract product description
        desc_match = re.search(r'Product Description:\s*(.*?)(?=Value:|$)', text, re.DOTALL)
        if desc_match:
            result['product_description'] = desc_match.group(1).strip()
        
        # ===== CORRECTED VALUE/UNIT EXTRACTION SECTION =====
        # Enhanced value/unit extraction - NOW PROPERLY INSIDE THE FUNCTION
        value_unit_patterns = [
            # Standard pattern: "Value: 1.5 Unit: count"
            r'Value:\s*([0-9.,]+)\s*Unit:\s*([a-zA-Z0-9\s]+)',
            # Compact pattern: "Value:1.5Unit:count"
            r'Value:\s*([0-9.,]+)\s*Unit:\s*([a-zA-Z0-9\s]+)',
            # Quantity pattern: "16 oz", "10 Count"
            r'(\d+(?:\.\d+)?)\s*(fl\s*oz|fl_oz|fluid\s*ounce|ounce|oz|count|ct|item|ml|g)\b',
            # Product description pattern: "Product Description: ... 10.5 oz ..."
            r'Product\s*Description:.*?(\d+(?:\.\d+)?)\s*(fl\s*oz|fl_oz|fluid\s*ounce|ounce|oz|count|ct|item|ml|g)\b',
            # Bullet point pattern: "Bullet Point: ... 10.5 oz ..."
            r'Bullet\s*Point.*?(\d+(?:\.\d+)?)\s*(fl\s*oz|fl_oz|fluid\s*ounce|ounce|oz|count|ct|item|ml|g)\b'
        ]

        for i, pattern in enumerate(value_unit_patterns):
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                groups = match.groups()
                if len(groups) >= 2:
                    value_str, unit_str = groups[0], groups[1]
                    # Clean value string
                    value_str = re.sub(r'[^\d.]', '', value_str)
                    try:
                        result['value'] = float(value_str)
                        result['unit'] = unit_str.strip()
                        if i > 1:  # Not one of the first two standard patterns
                            context.add_warning(PreprocessingStep.PARSING, 
                                              f"Used pattern #{i+1} for value/unit extraction", 
                                              sample_id)
                        break
                    except (ValueError, TypeError) as e:
                        context.add_warning(PreprocessingStep.PARSING, 
                                          f"Error converting value '{value_str}': {str(e)}", 
                                          sample_id)
                else:
                    context.add_warning(PreprocessingStep.PARSING, 
                                      f"Pattern matched but didn't capture both groups: {pattern}", 
                                      sample_id)
        
        # Final validation
        if result['value'] is not None and (result['value'] <= 0 or np.isinf(result['value'])):
            context.add_warning(PreprocessingStep.PARSING, 
                              f"Invalid value detected: {result['value']}", 
                              sample_id)
            result['value'] = None
            
        return result
    
    except Exception as e:
        context.add_error(PreprocessingStep.PARSING, e, sample_id)
        # Return minimal safe structure
        return {
            'item_name': '',
            'bullet_points': [],
            'product_description': '',
            'value': None,
            'unit': None,
            'raw_text': text
        }

@time_execution
def standardize_units(parsed_data: Dict[str, Any], context: PreprocessingContext, 
                      sample_id: Optional[int] = None) -> Dict[str, Any]:
    """
    Enhanced unit standardization with better fluid/dry distinction
    """
    result = parsed_data.copy()
    
    try:
        # Skip if no value/unit data
        if result['value'] is None or result['unit'] is None:
            context.record_stat('unit_conversion_failures')
            # Try to extract from item name as fallback
            if 'item_name' in result:
                quantity_match = re.search(r'(\d+(?:\.\d+)?)\s*(fl\s*oz|fl_oz|fluid\s*ounce|ounce|oz|count|ct|item|ml|g)', 
                                         result['item_name'], re.IGNORECASE)
                if quantity_match:
                    value_str, unit_str = quantity_match.groups()
                    try:
                        result['value'] = float(value_str)
                        result['unit'] = unit_str.lower()
                        context.add_warning(PreprocessingStep.UNIT_STANDARDIZATION, 
                                          f"Extracted unit from item name: {unit_str}", 
                                          sample_id)
                    except (ValueError, TypeError):
                        pass
            
            if result['value'] is None or result['unit'] is None:
                context.add_warning(PreprocessingStep.UNIT_STANDARDIZATION, 
                                  "Missing value or unit for standardization", 
                                  sample_id)
                result['std_value'] = None
                result['std_unit'] = None
                result['unit_category'] = None
                return result
        
        # Clean and normalize unit string
        unit = str(result['unit']).lower().strip()
        
        # Handle fluid indicators properly
        is_fluid = False
        if 'fl' in unit or 'fluid' in unit:
            is_fluid = True
            # Remove fluid indicator to get base unit
            unit = re.sub(r'fl\s*[-_]?\s*|fluid\s+', '', unit).strip()
        
        # Map to standard unit format
        std_value = None
        std_unit = None
        unit_category = None
        
        # Handle common unit variants
        unit_map = {
            'oz': 'oz', 'ounce': 'oz', 'ounces': 'oz',
            'fl_oz': 'fl_oz', 'fluid_ounce': 'fl_oz', 'fluid_ounces': 'fl_oz',
            'ml': 'ml', 'milliliter': 'ml', 'milliliters': 'ml',
            'l': 'l', 'liter': 'l', 'liters': 'l',
            'g': 'g', 'gram': 'g', 'grams': 'g',
            'kg': 'kg', 'kilogram': 'kg', 'kilograms': 'kg',
            'count': 'count', 'ct': 'count', 'item': 'count', 'items': 'count'
        }
        
        # Find matching standard unit
        base_unit = None
        for key in unit_map:
            if re.search(rf'\b{key}\b', unit):
                base_unit = unit_map[key]
                break
        
        # If no match found, try partial matches
        if not base_unit:
            for key in unit_map:
                if key in unit:
                    base_unit = unit_map[key]
                    break
        
        # Process based on base unit
        if base_unit:
            value = float(result['value'])
            
            # Fluid measurements
            if is_fluid or base_unit in ['fl_oz', 'ml', 'l']:
                std_unit = 'ml'
                unit_category = 'volume'
                if base_unit == 'fl_oz' or base_unit == 'oz':
                    std_value = value * 29.5735  # fl oz to ml
                elif base_unit == 'ml':
                    std_value = value
                elif base_unit == 'l':
                    std_value = value * 1000.0
            
            # Weight measurements
            elif base_unit in ['oz', 'g', 'kg', 'lb', 'pound']:
                std_unit = 'g'
                unit_category = 'weight'
                if base_unit == 'oz':
                    std_value = value * 28.3495  # oz to g
                elif base_unit == 'g':
                    std_value = value
                elif base_unit == 'kg':
                    std_value = value * 1000.0
                elif base_unit in ['lb', 'pound']:
                    std_value = value * 453.592
            
            # Count measurements
            elif base_unit == 'count':
                std_unit = 'item'
                unit_category = 'count'
                std_value = value
        
        # Store results
        result['std_value'] = std_value
        result['std_unit'] = std_unit
        result['unit_category'] = unit_category
        
        # Validate result
        if std_value is None or std_value <= 0:
            context.add_warning(PreprocessingStep.UNIT_STANDARDIZATION, 
                              f"Invalid standardized value: {std_value} (unit: {unit})", 
                              sample_id)
            result['std_value'] = None
            result['std_unit'] = None
            result['unit_category'] = None
        
        return result
    
    except Exception as e:
        context.add_error(PreprocessingStep.UNIT_STANDARDIZATION, e, sample_id)
        result['std_value'] = None
        result['std_unit'] = None
        result['unit_category'] = None
        return result

@time_execution
def extract_brand_info(item_name: str, bullet_points: List[str], 
                       context: PreprocessingContext, sample_id: Optional[int] = None) -> Dict[str, Any]:
    """
    Extract brand information from product name and bullet points.
    
    Uses multiple strategies to identify brand with confidence scoring.
    
    Args:
        item_name: Product name string
        bullet_points: List of bullet point strings
        context: Preprocessing context
        sample_id: Optional sample ID for error tracking
        
    Returns:
        Dictionary with brand information
    """
    try:
        combined_text = item_name + " " + " ".join(bullet_points)
        combined_text = sanitize_string(combined_text).lower()
        
        # Strategy 1: Direct brand matching
        best_match = None
        highest_score = 0
        
        for brand, score in BRAND_PREMIUM_SCORES.items():
            brand_lower = brand.lower()
            # Check for exact match
            if brand_lower in combined_text:
                # Higher score for matches in item name vs bullet points
                position_score = 1.2 if brand_lower in item_name.lower() else 1.0
                match_score = score * position_score
                
                if match_score > highest_score:
                    highest_score = match_score
                    best_match = brand
        
        # Strategy 2: Pattern-based detection for common brand formats
        if not best_match:
            # Look for patterns like "Brand: X", "By X", etc.
            brand_patterns = [
                r'brand:\s*([a-zA-Z0-9\s]+?)(?:,|\.|\s|$)',
                r'by\s+([a-zA-Z0-9\s]+?)(?:,|\.|\s|$)',
                r'made by\s+([a-zA-Z0-9\s]+?)(?:,|\.|\s|$)'
            ]
            
            for pattern in brand_patterns:
                match = re.search(pattern, combined_text)
                if match:
                    potential_brand = match.group(1).strip()
                    # Check if this potential brand matches any known brands
                    for brand in BRAND_PREMIUM_SCORES:
                        if potential_brand in brand.lower():
                            best_match = brand
                            highest_score = BRAND_PREMIUM_SCORES[brand] * 0.8  # Lower confidence
                            break
                    if best_match:
                        break
        
        # Strategy 3: First word detection for generic products
        if not best_match:
            first_word = item_name.split()[0].strip().rstrip(':,."\'')
            if len(first_word) > 2 and first_word[0].isupper():
                # Likely a brand name (first word capitalized)
                best_match = first_word
                highest_score = 0.5  # Low confidence
        
        # Finalize results
        if best_match:
            context.record_stat('brand_detection_count', best_match)
            return {
                'brand': best_match,
                'brand_score': highest_score,
                'is_premium_brand': highest_score > 0.7,
                'brand_category': 'premium' if highest_score > 0.7 else 'standard'
            }
        else:
            return {
                'brand': 'Generic',
                'brand_score': 0.0,
                'is_premium_brand': False,
                'brand_category': 'generic'
            }
    
    except Exception as e:
        context.add_error(PreprocessingStep.BRAND_EXTRACTION, e, sample_id)
        return {
            'brand': 'Unknown',
            'brand_score': 0.0,
            'is_premium_brand': False,
            'brand_category': 'unknown'
        }

@time_execution
def detect_premium_attributes(text: str, context: PreprocessingContext, 
                              sample_id: Optional[int] = None) -> Dict[str, Any]:
    """
    Detect premium attributes in product description.
    
    Analyzes text for keywords indicating premium quality, health attributes, etc.
    
    Args:
        text: Text to analyze (item name + bullet points + description)
        context: Preprocessing context
        sample_id: Optional sample ID for error tracking
        
    Returns:
        Dictionary with premium attribute information
    """
    try:
        text = sanitize_string(text).lower()
        attributes = {
            'premium_keywords': [],
            'premium_score': 0.0,
            'health_attributes': [],
            'health_score': 0.0,
            'convenience_features': [],
            'convenience_score': 0.0,
            'sustainability_features': [],
            'sustainability_score': 0.0,
            'total_premium_score': 0.0
        }
        
        # Check for each keyword
        for keyword, score in PREMIUM_KEYWORDS.items():
            if keyword in text:
                # Categorize the keyword
                if keyword in ['organic', 'premium', 'gourmet', 'artisan', 'handcrafted', 
                              'luxury', 'signature', 'special reserve', 'premium quality', 
                              'craft', 'small batch', 'limited edition']:
                    attributes['premium_keywords'].append(keyword)
                    attributes['premium_score'] += score
                elif keyword in ['gluten free', 'non-gmo', 'vegan', 'keto', 'dairy free', 
                                'sugar free', 'kosher', 'halal', 'non dairy', 
                                'plant based', 'all natural', 'natural']:
                    attributes['health_attributes'].append(keyword)
                    attributes['health_score'] += score
                elif keyword in ['convenient', 'easy to prepare', 'ready in minutes',
                                'microwave ready', 'no refrigeration', 'lunchbox ready']:
                    attributes['convenience_features'].append(keyword)
                    attributes['convenience_score'] += score
                elif keyword in ['sustainable', 'eco friendly', 'biodegradable', 'recyclable']:
                    attributes['sustainability_features'].append(keyword)
                    attributes['sustainability_score'] += score
        
        # Calculate total score with diminishing returns
        attributes['total_premium_score'] = (
            min(attributes['premium_score'], 3.0) * 0.4 +
            min(attributes['health_score'], 3.0) * 0.3 +
            min(attributes['convenience_score'], 2.0) * 0.2 +
            min(attributes['sustainability_score'], 2.0) * 0.1
        )
        
        return attributes
    
    except Exception as e:
        context.add_error(PreprocessingStep.PREMIUM_ATTRIBUTES, e, sample_id)
        return {
            'premium_keywords': [],
            'premium_score': 0.0,
            'health_attributes': [],
            'health_score': 0.0,
            'convenience_features': [],
            'convenience_score': 0.0,
            'sustainability_features': [],
            'sustainability_score': 0.0,
            'total_premium_score': 0.0
        }

@time_execution
def assign_product_category(item_name: str, bullet_points: List[str], 
                            context: PreprocessingContext, sample_id: Optional[int] = None) -> str:
    """
    Assign product to a category based on name and description.
    
    Uses multiple strategies with confidence scoring to determine category.
    
    Args:
        item_name: Product name string
        bullet_points: List of bullet point strings
        context: Preprocessing context
        sample_id: Optional sample ID for error tracking
        
    Returns:
        Product category string
    """
    try:
        combined_text = item_name + " " + " ".join(bullet_points)
        combined_text = sanitize_string(combined_text).lower()
        
        # Category keywords with weights
        category_keywords = {
            'beverages': [
                ('beverage', 1.0), ('drink', 1.0), ('soda', 1.2), ('water', 0.8),
                ('juice', 1.0), ('coffee', 1.0), ('tea', 1.0), ('sauce', 0.5),
                ('soup', 0.5), ('stock', 0.5), ('dashi', 0.7), ('gravy', 0.7)
            ],
            'snacks': [
                ('snack', 1.0), ('candy', 1.2), ('chocolate', 1.0), ('cookie', 1.0),
                ('cracker', 0.8), ('chip', 0.7), ('popcorn', 1.0), ('nut', 0.9),
                ('trail mix', 1.1), ('energy bar', 1.0), ('granola', 0.8)
            ],
            'cereal': [
                ('cereal', 1.5), ('flakes', 1.2), ('oat', 0.9), ('muesli', 1.0),
                ('granola', 0.7), ('breakfast', 0.8), ('porridge', 1.0)
            ],
            'condiments': [
                ('sauce', 1.0), ('gravy', 1.2), ('ketchup', 1.0), ('mustard', 1.0),
                ('mayo', 0.8), ('dressing', 1.0), ('seasoning', 1.1), ('spice', 0.9),
                ('salt', 0.7), ('pepper', 0.7), ('oil', 0.6)
            ],
            'baked_goods': [
                ('cake', 1.0), ('muffin', 1.2), ('bread', 0.9), ('pastry', 1.0),
                ('cookie', 0.8), ('bar', 0.7), ('pie', 0.8), ('dough', 0.7)
            ],
            'personal_care': [
                ('soap', 1.0), ('shampoo', 1.2), ('lotion', 1.0), ('deodorant', 1.1),
                ('cosmetic', 1.0), ('makeup', 0.9), ('lip', 0.8), ('skin', 0.7),
                ('body wash', 1.2)
            ],
            'kitchenware': [
                ('bowl', 1.0), ('plate', 0.9), ('utensil', 1.0), ('cookware', 1.1),
                ('cutlery', 0.8), ('dish', 0.7), ('container', 0.9), ('jar', 0.7)
            ]
        }
        
        # Calculate scores for each category
        category_scores = {category: 0.0 for category in category_keywords}
        
        for category, keywords in category_keywords.items():
            for keyword, weight in keywords:
                if keyword in combined_text:
                    # Higher weight if keyword appears in item name
                    position_weight = 1.2 if keyword in item_name.lower() else 1.0
                    category_scores[category] += weight * position_weight
        
        # Get the category with highest score
        best_category = max(category_scores, key=category_scores.get)
        highest_score = category_scores[best_category]
        
        # If no strong match, use 'other'
        if highest_score < 0.5:
            best_category = 'other'
        
        context.record_stat('category_distribution', best_category)
        return best_category
    
    except Exception as e:
        context.add_error(PreprocessingStep.CATEGORY_ASSIGNMENT, e, sample_id)
        return 'other'

# ======================
# MISSING VALUE HANDLING
# ======================

@time_execution
def handle_missing_values(row: pd.Series, context: PreprocessingContext, 
                          training_data: bool = True, prices: Optional[pd.Series] = None) -> pd.Series:
    """
    Handle missing values with category-specific strategies.
    
    Args:
        row: Single row of processed data
        context: Preprocessing context
        training_data: Whether this is training data (has price labels)
        prices: Price series for training data (for imputation)
        
    Returns:
        Row with missing values handled
    """
    result = row.copy()
    
    try:
        # Handle missing value/unit
        if pd.isna(result.get('value', None)) or pd.isna(result.get('unit', None)):
            context.record_stat('missing_value_count')
            
            if training_data and prices is not None and 'price' in row:
                # For training data with price, we can calculate implied unit price
                try:
                    # Get category median price per standard unit
                    category = result.get('category', 'other')
                    category_data = context.category_data.get(category, {})
                    
                    if category_data and 'median_price_per_unit' in category_data:
                        implied_value = row['price'] / category_data['median_price_per_unit']
                        result['value'] = implied_value
                        result['unit'] = category_data['std_unit']
                        result['std_value'] = implied_value
                        result['std_unit'] = category_data['std_unit']
                        context.add_warning(PreprocessingStep.MISSING_VALUE_HANDLING,
                                          "Imputed value/unit from price and category data",
                                          row.get('sample_id', None))
                except (KeyError, TypeError, ZeroDivisionError):
                    pass
            
            # If still missing, use category defaults
            if pd.isna(result.get('value', None)) or pd.isna(result.get('unit', None)):
                category = result.get('category', 'other')
                category_info = PRODUCT_CATEGORIES.get(category, PRODUCT_CATEGORIES['other'])
                
                # Use median quantity for the category
                result['value'] = 1.0  # Default to single unit
                result['unit'] = category_info['unit']
                result['std_value'] = 1.0
                result['std_unit'] = category_info['std_unit']
                context.add_warning(PreprocessingStep.MISSING_VALUE_HANDLING,
                                  f"Used category default for missing value/unit (category: {category})",
                                  row.get('sample_id', None))
        
        # Handle missing brand
        if not result.get('brand') or result['brand'] in ['Unknown', 'Generic']:
            category = result.get('category', 'other')
            
            # For certain categories, we can infer brand from context
            if category in ['beverages', 'snacks', 'cereal']:
                result['brand'] = 'Generic_Food'
                result['brand_score'] = 0.2
                result['is_premium_brand'] = False
                result['brand_category'] = 'generic'
            elif category == 'personal_care':
                result['brand'] = 'Generic_Care'
                result['brand_score'] = 0.2
                result['is_premium_brand'] = False
                result['brand_category'] = 'generic'
            
            context.add_warning(PreprocessingStep.MISSING_VALUE_HANDLING,
                              f"Inferred generic brand for category: {category}",
                              row.get('sample_id', None))
        
        return result
    
    except Exception as e:
        context.add_error(PreprocessingStep.MISSING_VALUE_HANDLING, e, row.get('sample_id', None))
        return row  # Return original row on error

# ======================
# FEATURE ENGINEERING
# ======================

@time_execution
def engineer_features(processed_data: pd.DataFrame, context: PreprocessingContext) -> pd.DataFrame:
    """
    Create engineered features from parsed product data.
    
    Args:
        processed_data: DataFrame with parsed product data
        context: Preprocessing context
        
    Returns:
        DataFrame with engineered features
    """
    result = processed_data.copy()
    
    try:
        # Price per unit features (for training data only)
        if 'price' in result.columns:
            # Calculate price per standard unit
            result['price_per_std_unit'] = np.nan
            valid_mask = (result['std_value'] > 0) & ~result['std_value'].isna()
            
            if valid_mask.any():
                result.loc[valid_mask, 'price_per_std_unit'] = (
                    result.loc[valid_mask, 'price'] / result.loc[valid_mask, 'std_value']
                )
                
                # Calculate category-specific price per unit statistics
                for category in result['category'].unique():
                    cat_mask = (result['category'] == category) & valid_mask
                    if cat_mask.sum() > 0:
                        cat_prices = result.loc[cat_mask, 'price_per_std_unit']
                        context.category_data[category] = {
                            'median_price_per_unit': cat_prices.median(),
                            'mean_price_per_unit': cat_prices.mean(),
                            'std_price_per_unit': cat_prices.std(),
                            'count': len(cat_prices)
                        }
        
        # Unit-based features
        result['is_count_based'] = result['unit_category'] == 'count'
        result['is_weight_based'] = result['unit_category'] == 'weight'
        result['is_volume_based'] = result['unit_category'] == 'volume'
        
        # Brand features
        result['brand_premium_score'] = result['brand_score']
        result['is_premium_brand'] = result['is_premium_brand'].astype(int)
        
        # Premium attribute features
        result['total_premium_score'] = result['total_premium_score']
        result['has_health_attributes'] = result['health_attributes'].apply(lambda x: 1 if x else 0)
        result['health_attribute_count'] = result['health_attributes'].apply(len)
        result['premium_attribute_count'] = result['premium_keywords'].apply(len)
        
        # Category features
        result = pd.get_dummies(result, columns=['category'], prefix='category')
        
        # Text length features
        result['item_name_length'] = result['item_name'].apply(len)
        result['total_bullet_points'] = result['bullet_points'].apply(len)
        result['total_description_length'] = (
            result['item_name'].str.len() + 
            result['bullet_points'].apply(lambda x: sum(len(p) for x in x)) +
            result['product_description'].str.len()
        )
        
        # Package size features
        result['is_multi_pack'] = result['value'] > 1.0
        result['package_size_log'] = np.log1p(result['value'])
        
        # Price-related features (for training)
        if 'price' in result.columns:
            # Log-transformed price (often better for regression)
            result['log_price'] = np.log1p(result['price'])
            
            # Price buckets (for potential ensemble approaches)
            result['price_bucket'] = pd.qcut(result['price'], q=10, labels=False, duplicates='drop')
        
        return result
    
    except Exception as e:
        context.add_error(PreprocessingStep.FEATURE_ENGINEERING, e)
        return processed_data  # Return original on error

# ======================
# MAIN PREPROCESSING PIPELINE
# ======================

@validate_input_data
@time_execution
def preprocess_data(df: pd.DataFrame, is_training: bool = True, 
                    max_workers: int = 4) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Main preprocessing pipeline for the Amazon ML Challenge dataset.
    
    Args:
        df: Input DataFrame with at least 'sample_id' and 'catalog_content' columns
        is_training: Whether this is training data (has price labels)
        max_workers: Number of threads for parallel processing
        
    Returns:
        Tuple of (processed DataFrame, context summary)
    """
    # Validate competition constraints
    validate_competition_constraints()
    
    # Create preprocessing context
    context = create_preprocessing_context(df, is_training)
    
    logger.info(f"Starting preprocessing for {len(df)} records (training={is_training})")
    
    # Initialize results list
    processed_rows = []
    
    # Extract prices for training data (for imputation)
    prices = df['price'] if is_training and 'price' in df.columns else None
    
    # Process rows in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_idx = {
            executor.submit(process_single_row, df.iloc[i], context, is_training, prices): i 
            for i in range(len(df))
        }
        
        # Process completed tasks
        for future in as_completed(future_to_idx):
            idx = future_to_idx[future]
            try:
                processed_row = future.result()
                processed_rows.append(processed_row)
            except Exception as e:
                logger.error(f"Error processing row {idx}: {str(e)}")
                # Add minimal fallback row
                fallback_row = df.iloc[idx].copy()
                fallback_row['processing_error'] = True
                processed_rows.append(fallback_row)
    
    # Convert to DataFrame
    processed_df = pd.DataFrame(processed_rows)
    
    # Engineer additional features
    if not processed_df.empty:
        processed_df = engineer_features(processed_df, context)
    
    # Final validation
    validate_output(processed_df, context, is_training)
    
    # Get summary
    summary = context.get_summary()
    logger.info(f"Preprocessing completed. Warnings: {summary['warning_count']}, Errors: {summary['error_count']}")
    
    return processed_df, summary

@robust_retry(max_attempts=2)
def process_single_row(row: pd.Series, context: PreprocessingContext, 
                       is_training: bool, prices: Optional[pd.Series]) -> Dict[str, Any]:
    """Process a single row of data with comprehensive error handling."""
    result = row.to_dict()
    sample_id = row.get('sample_id', None)
    
    try:
        # 1. Parse catalog_content
        context.steps_completed.append(PreprocessingStep.PARSING)
        parsed = parse_catalog_content(row['catalog_content'], context, sample_id)
        result.update(parsed)
        
        # 2. Standardize units
        context.steps_completed.append(PreprocessingStep.UNIT_STANDARDIZATION)
        standardized = standardize_units(parsed, context, sample_id)
        result.update(standardized)
        
        # 3. Extract brand information
        context.steps_completed.append(PreprocessingStep.BRAND_EXTRACTION)
        brand_info = extract_brand_info(
            result['item_name'], 
            result['bullet_points'], 
            context, 
            sample_id
        )
        result.update(brand_info)
        
        # 4. Detect premium attributes
        context.steps_completed.append(PreprocessingStep.PREMIUM_ATTRIBUTES)
        combined_text = (
            result['item_name'] + " " + 
            " ".join(result['bullet_points']) + " " + 
            result['product_description']
        )
        premium_attrs = detect_premium_attributes(combined_text, context, sample_id)
        result.update(premium_attrs)
        
        # 5. Assign product category
        context.steps_completed.append(PreprocessingStep.CATEGORY_ASSIGNMENT)
        category = assign_product_category(
            result['item_name'], 
            result['bullet_points'], 
            context, 
            sample_id
        )
        result['category'] = category
        
        # 6. Handle missing values
        if 'price' in row and not pd.isna(row['price']):
            context.steps_completed.append(PreprocessingStep.MISSING_VALUE_HANDLING)
            result_series = pd.Series(result)
            result_series = handle_missing_values(
                result_series, 
                context, 
                training_data=True,
                prices=prices
            )
            result = result_series.to_dict()
        
        return result
    
    except Exception as e:
        context.add_error(PreprocessingStep.PARSING, e, sample_id)
        # Return minimal safe structure
        return {
            'sample_id': sample_id,
            'catalog_content': row.get('catalog_content', ''),
            'parsing_error': True,
            'error_message': str(e)
        }

def validate_output(df: pd.DataFrame, context: PreprocessingContext, is_training: bool) -> None:
    """Validate the final processed output meets requirements."""
    # Check for required columns - these should exist AFTER feature engineering
    required_cols = ['sample_id', 'std_value', 'std_unit']
    
    # Check if we have category-related features (one-hot encoded)
    category_features = [col for col in df.columns if col.startswith('category_')]
    if not category_features:
        context.add_warning(PreprocessingStep.FEATURE_ENGINEERING,
                          "No category one-hot encoded features were created")
    
    # Check for other critical engineered features
    critical_features = [
        'brand_score', 'total_premium_score', 'item_name_length',
        'is_weight_based', 'is_volume_based', 'is_count_based'
    ]
    
    missing = [col for col in required_cols if col not in df.columns]
    missing += [col for col in critical_features if col in required_cols and col not in df.columns]
    
    if missing:
        raise DataValidationError(f"Missing required columns in output: {missing}")
    
    # Check for NaN in critical fields
    critical_fields = ['std_value', 'std_unit']
    for field in critical_fields:
        if field in df.columns and df[field].isna().any():
            nan_count = df[field].isna().sum()
            context.add_warning(PreprocessingStep.PARSING, 
                              f"{nan_count} NaN values in critical field: {field}")
    
    # For training data, verify price is positive
    if is_training and 'price' in df.columns:
        negative_prices = (df['price'] <= 0).sum()
        if negative_prices > 0:
            context.validation_failures += negative_prices
            raise DataValidationError(f"Found {negative_prices} non-positive prices in training data")
    
    # Verify row count matches input
    if len(df) != context.stats['total_records']:
        context.validation_failures += 1
        raise DataValidationError(
            f"Row count mismatch: expected {context.stats['total_records']}, got {len(df)}"
        )

# ======================
# UTILITY FUNCTIONS FOR COMPETITION
# ======================

def save_preprocessed_data(df: pd.DataFrame, output_path: str, is_training: bool = False) -> None:
    """
    Save preprocessed data with appropriate columns.
    
    Args:
        df: Processed DataFrame
        output_path: Path to save the output CSV
        is_training: Whether this is training data (contains price)
    """
    # Always keep sample_id
    feature_columns = [
        'sample_id', 'std_value', 'std_unit', 'unit_category',
        'brand_score', 'is_premium_brand', 'total_premium_score',
        'health_attribute_count', 'premium_attribute_count', 'is_count_based',
        'is_weight_based', 'is_volume_based', 'item_name_length', 
        'total_bullet_points', 'is_multi_pack', 'package_size_log'
    ]
    
    # Add price column if it exists (for training data)
    if is_training and 'price' in df.columns:
        feature_columns.insert(1, 'price')
    
    # Filter columns, handling missing ones gracefully
    save_cols = [col for col in feature_columns if col in df.columns]
    result = df[save_cols].copy()
    
    # Create output directory if needed
    os.makedirs(os.path.dirname(os.path.abspath(output_path)), exist_ok=True)
    
    # Save to CSV
    result.to_csv(output_path, index=False)
    logger.info(f"Preprocessed data saved to {output_path} with {len(result)} records")

def generate_sample_submission(test_df: pd.DataFrame, output_path: str) -> None:
    """
    Generate a sample submission file with random but realistic prices.
    
    Args:
        test_df: Test DataFrame with sample_id
        output_path: Path to save the submission CSV
    """
    # Create realistic price distribution based on training patterns
    np.random.seed(42)
    
    # Generate prices with category-based distributions
    prices = []
    for _, row in test_df.iterrows():
        category = row.get('category', 'other')
        cat_info = PRODUCT_CATEGORIES.get(category, PRODUCT_CATEGORIES['other'])
        
        # Generate price with category-specific parameters
        price = max(0.1, np.random.normal(cat_info['median_price'], cat_info['std_dev']))
        prices.append(price)
    
    # Create submission DataFrame
    submission = pd.DataFrame({
        'sample_id': test_df['sample_id'],
        'price': prices
    })
    
    # Save to CSV
    submission.to_csv(output_path, index=False)
    logger.info(f"Sample submission generated with {len(submission)} records at {output_path}")

# ======================
# MAIN EXECUTION
# ======================

if __name__ == "__main__":
    # Example usage
    try:
        # Load sample data
        train_df = pd.read_csv("68e8d1d70b66d_student_resource\student_resource\dataset\sample_test.csv")
        
        # Preprocess training data
        processed_train, context_summary = preprocess_data(train_df, is_training=True)
        
        # Save preprocessed data
        save_preprocessed_data(processed_train, "68e8d1d70b66d_student_resource\student_resource\dataset\processed_train.csv", is_training=True)
        
        # Generate sample submission for test data
        test_df = pd.read_csv("68e8d1d70b66d_student_resource\student_resource\dataset\sample_test.csv")
        generate_sample_submission(test_df, "68e8d1d70b66d_student_resource\student_resource\dataset\sample_test_out.csv")
        
        # Save context summary for debugging
        with open("preprocessing_context.json", "w") as f:
            json.dump(context_summary, f, indent=2)
            
        logger.info("Preprocessing pipeline completed successfully")
        
    except Exception as e:
        logger.exception("Critical error in preprocessing pipeline")
        raise

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

# --- 1. Data Loading and Merging ---

def load_and_merge_data(feature_path='sample_test.csv', output_path='sample_test_out.csv'):
    """
    Loads the feature and output datasets from CSV files and merges them on 'sample_id'.
    """
    try:
        print("Loading data...")
        df_features = pd.read_csv(feature_path)
        df_output = pd.read_csv(output_path)
        df = pd.merge(df_features, df_output, on='sample_id', how='left')
        print(f"Data loaded successfully. Merged dataframe shape: {df.shape}")
        return df
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Please ensure the files are in the correct directory.")
        return None

# --- 2. NEW: Feature Extraction from Text ---

def extract_structured_data(text):
    """
    Extracts 'Value' and 'Unit' from the catalog_content string using regular expressions.
    
    Args:
        text (str): The raw text from the 'catalog_content' column.

    Returns:
        pandas.Series: A series containing the extracted value and unit.
    """
    if not isinstance(text, str):
        return pd.Series([np.nan, 'unknown'], index=['Value', 'Unit'])

    # Regex to find 'Value:' followed by a number (integer or float)
    value_match = re.search(r'Value:\s*([\d\.]+)', text)
    value = float(value_match.group(1)) if value_match else np.nan

    # Regex to find 'Unit:' followed by a word
    unit_match = re.search(r'Unit:\s*(\w+)', text)
    unit = unit_match.group(1) if unit_match else 'unknown'
    
    return pd.Series([value, unit], index=['Value', 'Unit'])


# --- 3. Text Cleaning and Normalization ---

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    """
    Applies a series of cleaning steps to a raw text string.
    """
    if not isinstance(text, str):
        return ""
    
    # Remove HTML tags and the Value/Unit lines to avoid them being treated as features
    text = re.sub(r'Value:.*', '', text)
    text = re.sub(r'Unit:.*', '', text)
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w) > 2]
    
    return ' '.join(lemmatized_words)

# --- 4. Structured Data Cleaning ---

def standardize_units(unit):
    """
    Standardizes unit strings to a consistent format.
    """
    if not isinstance(unit, str):
        return 'unknown'
    
    unit = unit.lower().strip()
    
    unit_map = {
        'oz': 'ounce', 'ounces': 'ounce',
        'fl oz': 'fl_oz', 'fz': 'fl_oz',
        'ct': 'count',
        'none': 'unknown'
    }
    
    return unit_map.get(unit, unit)

# --- 5. Main Preprocessing and Feature Engineering Pipeline ---

def create_feature_pipeline(df):
    """
    Creates and applies a full preprocessing pipeline to the dataframe.
    """
    if df is None:
        return None, None, None
        
    # --- A. NEW: Extract structured data first! ---
    print("\nExtracting 'Value' and 'Unit' from 'catalog_content'...")
    # This creates the 'Value' and 'Unit' columns that were previously missing.
    extracted_data = df['catalog_content'].apply(extract_structured_data)
    df = pd.concat([df, extracted_data], axis=1)
    print("Extraction complete. DataFrame now has 'Value' and 'Unit' columns.")
    
    # --- B. Prepare Target Variable (y) ---
    df.dropna(subset=['price'], inplace=True)
    y = np.log1p(df['price'])
    features_df = df.drop(columns=['price', 'sample_id', 'image_link'])

    # --- C. Clean and Prepare Features ---
    print("\nCleaning and preparing features...")
    # Now that Value/Unit are extracted, we can clean the text.
    features_df['cleaned_catalog'] = features_df['catalog_content'].apply(clean_text)
    features_df['Unit_standardized'] = features_df['Unit'].apply(standardize_units)
    # The 'Value' column already exists and is numeric, so no to_numeric is needed here.
    print("Feature preparation complete.")
    
    # --- D. Define Scikit-Learn Preprocessing Steps ---
    print("\nDefining scikit-learn preprocessing pipelines...")
    numeric_features = ['Value']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_features = ['Unit_standardized']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
    ])

    text_features_col = 'cleaned_catalog'
    text_transformer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

    # --- E. Create and Apply the Master Preprocessor ---
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features),
            ('text', text_transformer, text_features_col)
        ],
        remainder='drop',
        n_jobs=-1
    )

    print("\nApplying the full pipeline to the data...")
    X = preprocessor.fit_transform(features_df)
    
    print("\n--- Preprocessing Complete ---")
    print(f"Shape of the final feature matrix X: {X.shape}")
    print(f"Shape of the final target vector y: {y.shape}")
    
    return X, y, preprocessor


# --- Main execution block ---
if __name__ == '__main__':
    # Step 1: Load and merge the data
    df_merged = load_and_merge_data(feature_path='68e8d1d70b66d_student_resource\student_resource\dataset\sample_test.csv', output_path='68e8d1d70b66d_student_resource\student_resource\dataset\sample_test_out.csv')
    
    # Step 2: Run the full preprocessing pipeline (now with the fix)
    X_processed, y_processed, preprocessor_fitted = create_feature_pipeline(df_merged)
    
    if X_processed is not None:
        print("\n✅ Script executed successfully!")
        print("Variables `X_processed`, `y_processed`, and `preprocessor_fitted` are now ready.")
        print("\nNext steps:")
        print("1. Split data for training and validation: `X_train, X_test, y_train, y_test = train_test_split(...)`")
        print("2. Train a regression model: `model.fit(X_train, y_train)`")
        print("3. Remember to inverse transform predictions: `np.expm1(model.predict(X_test))`")

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy.sparse import save_npz # To save the sparse matrix
import joblib # To save the fitted preprocessor
import os
import warnings

warnings.filterwarnings('ignore')

# --- 1. Data Loading and Merging ---

def load_and_merge_data(feature_path='sample_test.csv', output_path='sample_test_out.csv'):
    """
    Loads the feature and output datasets from CSV files and merges them on 'sample_id'.
    """
    try:
        print("Loading data...")
        df_features = pd.read_csv(feature_path)
        df_output = pd.read_csv(output_path)
        df = pd.merge(df_features, df_output, on='sample_id', how='left')
        print(f"Data loaded successfully. Merged dataframe shape: {df.shape}")
        return df
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Please ensure the files are in the correct directory.")
        return None

# --- 2. Feature Extraction from Text ---

def extract_structured_data(text):
    """
    Extracts 'Value' and 'Unit' from the catalog_content string using regular expressions.
    """
    if not isinstance(text, str):
        return pd.Series([np.nan, 'unknown'], index=['Value', 'Unit'])

    value_match = re.search(r'Value:\s*([\d\.]+)', text)
    value = float(value_match.group(1)) if value_match else np.nan

    unit_match = re.search(r'Unit:\s*(\w+)', text)
    unit = unit_match.group(1) if unit_match else 'unknown'
    
    return pd.Series([value, unit], index=['Value', 'Unit'])

# --- 3. Text Cleaning and Normalization ---

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    """
    Applies a series of cleaning steps to a raw text string.
    """
    if not isinstance(text, str):
        return ""
    
    text = re.sub(r'Value:.*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Unit:.*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w) > 2]
    
    return ' '.join(lemmatized_words)

# --- 4. Structured Data Cleaning ---

def standardize_units(unit):
    """
    Standardizes unit strings to a consistent format.
    """
    if not isinstance(unit, str):
        return 'unknown'
    
    unit = unit.lower().strip()
    
    unit_map = {
        'oz': 'ounce', 'ounces': 'ounce',
        'fl oz': 'fl_oz', 'fz': 'fl_oz',
        'ct': 'count',
        'none': 'unknown'
    }
    
    return unit_map.get(unit, unit)

# --- 5. Main Preprocessing and Feature Engineering Pipeline ---

def create_feature_pipeline(df):
    """
    Creates and applies a full preprocessing pipeline to the dataframe.
    """
    if df is None:
        return None, None, None
        
    print("\nExtracting 'Value' and 'Unit' from 'catalog_content'...")
    extracted_data = df['catalog_content'].apply(extract_structured_data)
    df = pd.concat([df, extracted_data], axis=1)
    print("Extraction complete.")
    
    df.dropna(subset=['price'], inplace=True)
    y = np.log1p(df['price'])
    # Store sample_ids to align with y if needed later
    sample_ids = df['sample_id']
    features_df = df.drop(columns=['price', 'sample_id', 'image_link'])

    print("\nCleaning and preparing features...")
    features_df['cleaned_catalog'] = features_df['catalog_content'].apply(clean_text)
    features_df['Unit_standardized'] = features_df['Unit'].apply(standardize_units)
    print("Feature preparation complete.")
    
    print("\nDefining scikit-learn preprocessing pipelines...")
    numeric_features = ['Value']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_features = ['Unit_standardized']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
    ])

    text_features_col = 'cleaned_catalog'
    text_transformer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features),
            ('text', text_transformer, text_features_col)
        ],
        remainder='drop',
        n_jobs=-1
    )

    print("\nApplying the full pipeline to the data...")
    X = preprocessor.fit_transform(features_df)
    
    print("\n--- Preprocessing Complete ---")
    print(f"Shape of the final feature matrix X: {X.shape}")
    print(f"Shape of the final target vector y: {y.shape}")
    
    # Return sample_ids along with the other artifacts
    return X, y, sample_ids, preprocessor

# --- 6. Save Processed Data and Preprocessor ---

def save_processed_data(X, y, ids, preprocessor, output_dir='processed_data'):
    """
    Saves the processed feature matrix, target vector, and fitted preprocessor.

    Args:
        X (scipy.sparse.csr_matrix): The sparse feature matrix.
        y (pandas.Series): The target vector.
        ids (pandas.Series): The sample_ids corresponding to the target vector.
        preprocessor (sklearn.compose.ColumnTransformer): The fitted preprocessor object.
        output_dir (str): The directory to save the files in.
    """
    try:
        print(f"\nSaving processed data to directory: '{output_dir}'")
        os.makedirs(output_dir, exist_ok=True)
        
        # Save the sparse feature matrix
        save_npz(os.path.join(output_dir, 'X_processed.npz'), X)
        
        # Save the target vector and its corresponding IDs to a CSV
        y_df = pd.DataFrame({'sample_id': ids, 'price_log': y})
        y_df.to_csv(os.path.join(output_dir, 'y_processed.csv'), index=False)
        
        # Save the fitted preprocessor object for later use on the test set
        joblib.dump(preprocessor, os.path.join(output_dir, 'preprocessor.joblib'))
        
        print("All files saved successfully:")
        print(f"- {os.path.join(output_dir, 'X_processed.npz')}")
        print(f"- {os.path.join(output_dir, 'y_processed.csv')}")
        print(f"- {os.path.join(output_dir, 'preprocessor.joblib')}")
        
    except Exception as e:
        print(f"An error occurred while saving the data: {e}")

# --- Main execution block ---
if __name__ == '__main__':
    # Step 1: Load and merge the data
    df_merged = load_and_merge_data(feature_path='68e8d1d70b66d_student_resource\student_resource\dataset\sample_test.csv', output_path='68e8d1d70b66d_student_resource\student_resource\dataset\sample_test_out.csv')
    
    # Step 2: Run the full preprocessing pipeline
    X_processed, y_processed, sample_ids_processed, preprocessor_fitted = create_feature_pipeline(df_merged)
    
    # Step 3: Save the results for later use
    if X_processed is not None:
        save_processed_data(X_processed, y_processed, sample_ids_processed, preprocessor_fitted)

        print("\n✅ Script executed successfully!")
        print("\nNext steps:")
        print("1. In your training script, you can now load these files directly.")
        print("   - `from scipy.sparse import load_npz`")
        print("   - `X = load_npz('processed_data/X_processed.npz')`")
        print("   - `y_df = pd.read_csv('processed_data/y_processed.csv')`")
        print("   - `y = y_df['price_log']`")
        print("2. Train your model on this loaded data.")

"extract_text_from_images.py"

In [ ]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import os
import base64 # Required for encoding images
import json   # Required for constructing the JSON payload

# --- 1. API Interaction Function ---

def get_local_vlm_description(image_url: str) -> str:
    api_url = "http://localhost:1234/v1/chat/completions"

    # Strict OCR-style prompt
    prompt_text = (
        "Extract ONLY the exact text visible in the image. "
        "Do not describe the product, do not add explanations, "
        "do not summarize, and do not include anything extra. "
        "Return only the raw text exactly as it appears."
    )

    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(image_url, timeout=20, headers=headers)
        response.raise_for_status()

        base64_image = base64.b64encode(response.content).decode('utf-8')

        payload = {
            "model": "smolvlm2-500m-video-instruct",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_text},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 250,
            "temperature": 0.0  # deterministic, no creativity
        }

        api_response = requests.post(api_url, headers={"Content-Type": "application/json"}, json=payload)
        api_response.raise_for_status()
        response_json = api_response.json()

        if 'choices' in response_json and len(response_json['choices']) > 0:
            content = response_json['choices'][0]['message']['content']
            return content.strip()
        else:
            print(f"\nAPI returned an unexpected response for {image_url}: {response_json}")
            return ""

    except requests.exceptions.RequestException as e:
        print(f"\nNetwork or Download Error for {image_url}: {e}")
        return ""
    except Exception as e:
        print(f"\nAn unexpected error occurred for {image_url}: {e}")
        return ""

# --- Main execution block ---
if __name__ == '__main__':
    print("--- Starting Feature Extraction using Local LM Studio VLM ---")
    print("IMPORTANT: Ensure your LM Studio server is running with the correct model loaded.")

    # Path to your CSV file
    CSV_PATH = '68e8d1d70b66d_student_resource\student_resource\dataset\sample_test.csv' 
    OUTPUT_DIR = 'processed_data'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print(f"Loading CSV file: {CSV_PATH}...")
    df = pd.read_csv(CSV_PATH)
    df.drop_duplicates(subset=['sample_id', 'image_link'], inplace=True)

    local_vlm_descriptions = []
    print(f"\nGenerating descriptions for {len(df)} images via local API...")

    for _, row in tqdm(df.iterrows(), total=len(df)):
        description = get_local_vlm_description(row['image_link'])
        local_vlm_descriptions.append(description)

    # Save the results
    results_df = pd.DataFrame({
        'sample_id': df['sample_id'],
        'smolvlm_description': local_vlm_descriptions
    })

    output_path = os.path.join(OUTPUT_DIR, 'smolvlm_extracted_features_only.csv')
    results_df.to_csv(output_path, index=False)

    print(f"\n✅ Local VLM feature extraction complete. Descriptions saved to '{output_path}'")

In [ ]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import time
import warnings
warnings.filterwarnings('ignore')

# ==========================================
# GPU SETUP
# ==========================================

def setup_device():
    """Setup PyTorch device with optimization."""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        
        print(f"✅ GPU: {gpu_name}")
        print(f"   Memory: {gpu_memory:.1f} GB")
        print(f"   CUDA: {torch.version.cuda}")
        
        # Optimization for RTX 4060
        torch.backends.cudnn.benchmark = True  # Auto-tune convolutions
        torch.cuda.empty_cache()
        
    else:
        device = torch.device('cpu')
        print("⚠️  Using CPU")
    
    return device

# ==========================================
# MODEL SETUP
# ==========================================

def setup_model(device):
    """Load ResNet50 optimized for inference."""
    print("Loading ResNet50...")
    
    # Load pretrained model
    model = models.resnet50(weights='IMAGENET1K_V2')
    
    # Remove classification head (keep features only)
    model = torch.nn.Sequential(*list(model.children())[:-1])
    
    # Move to GPU and set to eval mode
    model = model.to(device)
    model.eval()
    
    # Enable mixed precision for faster inference (RTX 4060 supports this)
    if device.type == 'cuda':
        model = model.half()  # FP16 for 2x speedup
        print("✅ Model loaded with FP16 optimization (2048D features)")
    else:
        print("✅ Model loaded (2048D features)")
    
    return model

# ==========================================
# IMAGE PREPROCESSING
# ==========================================

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def download_image(url, max_retries=3):
    """Download and preprocess image with retries."""
    for attempt in range(max_retries):
        try:
            response = requests.get(
                url,
                timeout=12,
                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
            )
            response.raise_for_status()
            
            # Load and convert to RGB
            img = Image.open(BytesIO(response.content)).convert('RGB')
            
            # Apply transforms
            return transform(img)
            
        except Exception:
            if attempt < max_retries - 1:
                time.sleep(0.3)
            else:
                return None
    
    return None

# ==========================================
# BATCH PROCESSING
# ==========================================

def process_batch(urls, sample_ids, model, device, max_workers=20):
    """Process batch with parallel downloads + GPU inference."""
    
    # Parallel download
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        images = list(executor.map(download_image, urls))
    
    # Separate valid/invalid
    valid_images = []
    valid_indices = []
    
    for i, img in enumerate(images):
        if img is not None:
            valid_images.append(img)
            valid_indices.append(i)
    
    # Initialize with NaN
    features = np.full((len(urls), 2048), np.nan, dtype=np.float32)
    
    # GPU inference
    if valid_images:
        batch_tensor = torch.stack(valid_images).to(device)
        
        # Use FP16 if GPU
        if device.type == 'cuda':
            batch_tensor = batch_tensor.half()
        
        with torch.no_grad():
            batch_features = model(batch_tensor)
            batch_features = batch_features.squeeze(-1).squeeze(-1)
            batch_features = batch_features.float().cpu().numpy()  # Back to FP32
        
        # Assign features
        for i, idx in enumerate(valid_indices):
            features[idx] = batch_features[i]
    
    success = [img is not None for img in images]
    
    return sample_ids, features, success

# ==========================================
# CHECKPOINTING
# ==========================================

class Checkpoint:
    """Incremental saving with resume capability."""
    
    def __init__(self, output_dir):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
        self.features_file = os.path.join(output_dir, 'X_image_features.npy')
        self.ids_file = os.path.join(output_dir, 'all_sample_ids.csv')
    
    def load_progress(self):
        """Get already processed IDs."""
        if os.path.exists(self.ids_file):
            df = pd.read_csv(self.ids_file)
            processed = set(df['sample_id'].values)
            print(f"📂 Resuming: {len(processed)} processed")
            return processed
        return set()
    
    def save_batch(self, sample_ids, features):
        """Append batch."""
        # Features
        if os.path.exists(self.features_file):
            existing = np.load(self.features_file)
            combined = np.vstack([existing, features])
        else:
            combined = features
        
        np.save(self.features_file, combined)
        
        # IDs
        pd.DataFrame({'sample_id': sample_ids}).to_csv(
            self.ids_file,
            mode='a',
            header=not os.path.exists(self.ids_file),
            index=False
        )

# ==========================================
# MAIN PIPELINE
# ==========================================

def extract_features(
    train_csv='dataset/train.csv',
    test_csv='dataset/test.csv',
    output_dir='processed_data',
    batch_size=96,
    max_workers=20
):
    """Main extraction pipeline."""
    
    print("="*70)
    print("Amazon ML Challenge 2025 - Image Feature Extraction")
    print("="*70)
    
    # Setup
    device = setup_device()
    model = setup_model(device)
    checkpoint = Checkpoint(output_dir)
    
    # Load data
    print(f"\n📂 Loading data...")
    df_train = pd.read_csv(train_csv)
    df_test = pd.read_csv(test_csv)
    
    df_all = pd.concat([
        df_train[['sample_id', 'image_link']],
        df_test[['sample_id', 'image_link']]
    ]).drop_duplicates(subset=['sample_id']).reset_index(drop=True)
    
    print(f"   Total: {len(df_all)}")
    
    # Resume
    processed = checkpoint.load_progress()
    df_todo = df_all[~df_all['sample_id'].isin(processed)].reset_index(drop=True)
    
    if df_todo.empty:
        print("\n✅ All processed!")
        return
    
    print(f"   Remaining: {len(df_todo)}\n")
    
    # Process
    num_batches = (len(df_todo) + batch_size - 1) // batch_size
    
    print(f"🚀 Processing batches of {batch_size} (ETA: ~{num_batches*2.5/60:.0f} min)")
    print(f"   Workers: {max_workers}\n")
    
    start = time.time()
    stats = {'success': 0, 'failed': 0}
    
    pbar = tqdm(range(0, len(df_todo), batch_size), total=num_batches, desc="Progress")
    
    for i in pbar:
        batch_df = df_todo.iloc[i:i + batch_size]
        
        batch_ids, batch_features, success = process_batch(
            batch_df['image_link'].tolist(),
            batch_df['sample_id'].tolist(),
            model, device, max_workers
        )
        
        checkpoint.save_batch(batch_ids, batch_features)
        
        # Stats
        stats['success'] += sum(success)
        stats['failed'] += len(success) - sum(success)
        
        # Update progress bar
        total = stats['success'] + stats['failed']
        rate = stats['success'] / total * 100 if total > 0 else 0
        speed = total / (time.time() - start)
        pbar.set_postfix({
            'Success': f"{rate:.1f}%",
            'Speed': f"{speed:.1f} img/s"
        })
        
        # Clear GPU cache periodically
        if i % 50 == 0 and device.type == 'cuda':
            torch.cuda.empty_cache()
    
    elapsed = time.time() - start
    
    # Summary
    print(f"\n{'='*70}")
    print("✅ Extraction Complete!")
    print(f"{'='*70}")
    total = stats['success'] + stats['failed']
    print(f"Total: {total:,}")
    print(f"Success: {stats['success']:,} ({stats['success']/total*100:.1f}%)")
    print(f"Failed: {stats['failed']:,}")
    print(f"Time: {elapsed/60:.1f} min")
    print(f"Speed: {total/elapsed:.1f} images/sec")
    
    # Impute
    print(f"\n🔧 Final processing...")
    features = np.load(checkpoint.features_file)
    
    if np.isnan(features).any():
        n_missing = np.isnan(features).any(axis=1).sum()
        print(f"   Imputing {n_missing:,} samples...")
        
        col_means = np.nanmean(features, axis=0)
        for col in range(features.shape[1]):
            mask = np.isnan(features[:, col])
            features[mask, col] = col_means[col]
        
        np.save(checkpoint.features_file, features)
    
    print(f"\n✅ Saved: {checkpoint.features_file}")
    print(f"   Shape: {features.shape}")
    print(f"{'='*70}\n")

# ==========================================
# RUN
# ==========================================

if __name__ == '__main__':
    CONFIG = {
        'train_csv': 'dataset/train.csv',
        'test_csv': 'dataset/test.csv',
        'output_dir': 'processed_data',
        'batch_size': 96,      # Optimized for RTX 4060
        'max_workers': 20      # Max parallel downloads
    }
    
    extract_features(**CONFIG)


Amazon ML Challenge 2025 - Image Feature Extraction
✅ GPU: NVIDIA GeForce RTX 4060 Laptop GPU
   Memory: 8.6 GB
   CUDA: 11.8
Loading ResNet50...


✅ Model loaded with FP16 optimization (2048D features)

📂 Loading data...
   Total: 150000
📂 Resuming: 17312 processed
   Remaining: 132688

🚀 Processing batches of 96 (ETA: ~58 min)
   Workers: 20



Progress:   7%|▋         | 94/1383 [19:44<4:19:00, 12.06s/it, Success=100.0%, Speed=7.6 img/s]

In [3]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU count: {torch.cuda.device_count()}")


PyTorch version: 2.7.0+cu118
CUDA available: True
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU
GPU count: 1


In [ ]:
!pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, hstack
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb
from sklearn.metrics import r2_score
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os

# --- 1. Re-usable Components ---

def smape(y_true, y_pred):
    """Calculates the Symmetric Mean Absolute Percentage Error (SMAPE)."""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def clean_text(text):
    """A consistent cleaning function for both text sources."""
    if not isinstance(text, str): return ""
    text = re.sub(r'<.*?>|Value:.*|Unit:.*', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w) > 2]
    return ' '.join(lemmatized_words)

# --- 2. Load and Combine All Data Sources ---

def load_and_combine_all_features(data_dir='processed_data'):
    """
    Loads all preprocessed features and combines them.
    Returns the combined feature matrix and the original DataFrame with prices.
    """
    try:
        print("--- Loading All Preprocessed Data ---")
        
        X_text_struct = load_npz(f'{data_dir}/X_processed.npz')
        y_df = pd.read_csv(f'{data_dir}/y_processed.csv').set_index('sample_id')
        
        vlm_df = pd.read_csv(f'{data_dir}/smolvlm_extracted_features.csv').set_index('sample_id')
        vlm_df = vlm_df.reindex(y_df.index).fillna('')
        vlm_df['cleaned_vlm'] = vlm_df['smolvlm_description'].apply(clean_text)
        
        vlm_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2))
        X_vlm = vlm_vectorizer.fit_transform(vlm_df['cleaned_vlm'])
        
        X_images = np.load(f'{data_dir}/X_image_features.npy')
        all_ids_df = pd.read_csv(f'{data_dir}/all_sample_ids.csv').set_index('sample_id')
        image_features_df = pd.DataFrame(X_images, index=all_ids_df.index)
        X_images_aligned = image_features_df.reindex(y_df.index).values
        
        X_final_combined = hstack([X_text_struct, X_vlm, X_images_aligned]).tocsr()

        print(f"✅ Data loaded and combined successfully! Final shape: {X_final_combined.shape}")
        
        # We need the original dataframe with IDs and prices for splitting and scoring
        full_df = pd.read_csv(f'{data_dir}/y_processed.csv')
        print("Vectorizing VLM text...")
        vlm_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2))
        # This line was changed: we fit_transform on the training data
        X_vlm = vlm_vectorizer.fit_transform(vlm_df['cleaned_vlm'])
        
        # --- ADD THIS LINE ---
        joblib.dump(vlm_vectorizer, f'{data_dir}/vlm_vectorizer.joblib')
        print("Saved VLM TF-IDF vectorizer.")
        return X_final_combined, full_df
        
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Please ensure you have run all three preprocessing scripts first.")
        return None, None

# --- 3. Full Train, Predict, and Evaluate Pipeline ---

def run_full_pipeline(X, df):
    """
    Trains on a subset, predicts on a holdout, saves a submission file, and scores it.
    """
    if X is None or df is None: return

    print("\n--- Simulating Competition Workflow ---")
    
    # --- Step 1: Split the data into a training set and a "pretend" test set ---
    # We split based on the dataframe to keep track of IDs
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Get the corresponding rows from the sparse matrix X
    X_train = X[train_df.index]
    y_train = train_df['price_log']
    
    X_test = X[test_df.index]
    # We will use this later to score our predictions
    y_test_ground_truth_log = test_df['price_log'] 

    print(f"Training on {len(train_df)} samples, predicting on {len(test_df)} samples.")

    # --- Step 2: Train the model ONLY on the training set ---
    print("\nTraining a LightGBM Regressor model...")
    lgbm = lgb.LGBMRegressor(
        objective='regression_l1', metric='mae', n_estimators=1000,
        learning_rate=0.05, num_leaves=31, random_state=42, n_jobs=-1
    )
    lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test_ground_truth_log)], callbacks=[lgb.early_stopping(50, verbose=False)])
    print("Model training complete.")

    # --- Step 3: Make predictions on the "pretend" test set ---
    print("Making predictions on the holdout test set...")
    predictions_log = lgbm.predict(X_test)
    
    # Inverse transform to get actual prices
    predictions_actual = np.expm1(predictions_log)
    predictions_actual[predictions_actual < 0] = 0 # Enforce non-negative constraint

    # --- Step 4: Create and save the submission CSV file ---
    submission_df = pd.DataFrame({
        'sample_id': test_df['sample_id'],
        'price': predictions_actual
    })
    
    submission_path = 'sample_submission.csv'
    submission_df.to_csv(submission_path, index=False)
    print(f"✅ Submission file created at: '{submission_path}'")

    # --- Step 5: Score the submission file against the ground truth ---
    print("\n--- Scoring Submission File ---")
    
    # Load the ground truth from the original file
    ground_truth_df = pd.read_csv('68e8d1d70b66d_student_resource/student_resource/dataset/sample_test_out.csv')
    
    # Merge our predictions with the ground truth
    # This mimics exactly how the competition leaderboard is calculated
    merged_score_df = pd.merge(submission_df, ground_truth_df, on='sample_id', suffixes=('_pred', '_true'))
    
    if len(merged_score_df) != len(test_df):
        print("Warning: The number of predictions does not match the test set size!")

    # Calculate the final SMAPE score
    final_smape_score = smape(merged_score_df['price_true'], merged_score_df['price_pred'])
    
    print("\n--- FINAL SIMULATED SCORE ---")
    print(f"SMAPE against sample_test_out.csv: {final_smape_score:.4f}%")
    
    return lgbm

# --- Main execution block ---
if __name__ == '__main__':
    # Load all features and the corresponding dataframe with IDs and prices
    X_final, df_final = load_and_combine_all_features()
    
    # Run the full simulation
    final_model = run_full_pipeline(X_final, df_final)
    
    if final_model:
        joblib.dump(final_model, 'final_lgbm_model_for_submission.joblib')
        print("\n✅ Final trained model saved as 'final_lgbm_model_for_submission.joblib'")
        print("You can use this model to predict on the real 'test.csv' for the competition.")

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
import joblib
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import os # Added for path joining
from sklearn.metrics import r2_score # Added for R-squared score

# --- 1. Load All Necessary Artifacts and Helper Functions ---
print("--- Loading Artifacts for Prediction ---")

# --- Helper Functions (must be identical to training) ---
# NOTE: The SMAPE function is now defined here for scoring
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def clean_text(text):
    if not isinstance(text, str): return ""
    text = re.sub(r'<.*?>|Value:.*|Unit:.*', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w) > 2]
    return ' '.join(lemmatized_words)

def extract_structured_data(text):
    if not isinstance(text, str): return pd.Series([np.nan, 'unknown'], index=['Value', 'Unit'])
    value_match = re.search(r'Value:\s*([\d\.]+)', text)
    value = float(value_match.group(1)) if value_match else np.nan
    unit_match = re.search(r'Unit:\s*(\w+)', text)
    unit = unit_match.group(1) if unit_match else 'unknown'
    return pd.Series([value, unit], index=['Value', 'Unit'])

def standardize_units(unit):
    if not isinstance(unit, str): return 'unknown'
    unit = unit.lower().strip()
    unit_map = {'oz': 'ounce', 'ounces': 'ounce', 'fl oz': 'fl_oz', 'fz': 'fl_oz', 'ct': 'count', 'none': 'unknown'}
    return unit_map.get(unit, unit)

# --- Load Saved Objects ---
try:
    PREPROCESSOR_PATH = 'processed_data/preprocessor.joblib'
    VLM_VECTORIZER_PATH = 'processed_data/vlm_vectorizer.joblib'
    MODEL_PATH = 'final_lgbm_model_for_submission.joblib'
    
    preprocessor = joblib.load(PREPROCESSOR_PATH)
    vlm_vectorizer = joblib.load(VLM_VECTORIZER_PATH)
    model = joblib.load(MODEL_PATH)
    print("✅ All models and preprocessors loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading artifact: {e}")
    print("Please ensure you have run the full training pipeline on the complete dataset first.")
    exit()

# --- 2. Feature Engineering Function for New Data (using robust reindexing) ---
def create_features_for_test_data(df, data_dir='processed_data'):
    """
    Applies the complete feature engineering pipeline to new, unseen test data.
    """
    print("\n--- Applying Feature Engineering to Test Data ---")
    
    df_indexed = df.set_index('sample_id')

    print("Processing catalog content...")
    extracted_data = df['catalog_content'].apply(extract_structured_data)
    df_features = pd.concat([df, extracted_data], axis=1)
    df_features['cleaned_catalog'] = df_features['catalog_content'].apply(clean_text)
    df_features['Unit_standardized'] = df_features['Unit'].apply(standardize_units)
    X_text_struct = preprocessor.transform(df_features)

    print("Processing VLM text features...")
    vlm_df_all = pd.read_csv(os.path.join(data_dir, 'smolvlm_extracted_features.csv')).set_index('sample_id')
    vlm_df_aligned = vlm_df_all.reindex(df_indexed.index).fillna('')
    vlm_df_aligned['cleaned_vlm'] = vlm_df_aligned['smolvlm_description'].apply(clean_text)
    X_vlm = vlm_vectorizer.transform(vlm_df_aligned['cleaned_vlm'])

    print("Processing visual features...")
    all_image_features = np.load(os.path.join(data_dir, 'X_image_features.npy'))
    all_ids_df = pd.read_csv(os.path.join(data_dir, 'all_sample_ids.csv')).set_index('sample_id')
    image_features_df = pd.DataFrame(all_image_features, index=all_ids_df.index)
    image_features_aligned = image_features_df.reindex(df_indexed.index)
    
    if image_features_aligned.isnull().values.any():
        print("Warning: Missing visual features for some IDs. Imputing with mean.")
        image_features_aligned = image_features_aligned.fillna(image_features_aligned.mean())
    X_images_test = image_features_aligned.values

    print("Combining all feature sets...")
    X_test_final = hstack([X_text_struct, X_vlm, X_images_test]).tocsr()
    
    if X_text_struct.shape[0] != X_images_test.shape[0]:
         raise ValueError(f"Row count mismatch! Text features: {X_text_struct.shape[0]}, Image features: {X_images_test.shape[0]}")
    
    print(f"✅ Final test feature matrix created with shape: {X_test_final.shape}")
    return X_test_final

# --- 3. Main Prediction & Evaluation Block ---
if __name__ == '__main__':
    # --- Define file paths for the sample data evaluation ---
    # We will PREDICT on `sample_test.csv` and SCORE against `sample_test_out.csv`
    TEST_DATA_PATH = '68e8d1d70b66d_student_resource\student_resource\dataset\sample_test.csv'
    GROUND_TRUTH_PATH = '68e8d1d70b66d_student_resource\student_resource\dataset\sample_test_out.csv'
    SUBMISSION_FILE_PATH = 'sample_test_prediction_output.csv' # Give it a different name
    
    print(f"\nLoading SAMPLE test data from: {TEST_DATA_PATH}")
    test_df = pd.read_csv(TEST_DATA_PATH)
    
    # --- Create features for the test data ---
    X_test = create_features_for_test_data(test_df.copy())
    
    # --- Make Predictions ---
    print("\nMaking final predictions on sample data...")
    predictions_log = model.predict(X_test)
    
    predictions_actual = np.expm1(predictions_log)
    predictions_actual[predictions_actual < 0] = 0

    # --- Create and Save the Prediction File ---
    submission_df = pd.DataFrame({
        'sample_id': test_df['sample_id'],
        'price': predictions_actual
    })
    submission_df.to_csv(SUBMISSION_FILE_PATH, index=False)
    print(f"Prediction file for sample data generated at: '{SUBMISSION_FILE_PATH}'")
    
    # --- NEW: Score the predictions against the ground truth ---
    print("\n--- Scoring Predictions Against Ground Truth ---")
    try:
        ground_truth_df = pd.read_csv(GROUND_TRUTH_PATH)
        
        # Merge our predictions with the true prices
        merged_score_df = pd.merge(submission_df, ground_truth_df, on='sample_id', suffixes=('_pred', '_true'))
        
        if len(merged_score_df) != len(test_df):
            print("Warning: Not all sample IDs could be scored.")

        # Calculate and print the final scores
        final_smape_score = smape(merged_score_df['price_true'], merged_score_df['price_pred'])
        final_r2_score = r2_score(merged_score_df['price_true'], merged_score_df['price_pred'])

        print("\n--- FINAL PERFORMANCE ON SAMPLE DATA ---")
        print(f"SMAPE Score: {final_smape_score:.4f}%")
        print(f"R-squared (R²): {final_r2_score:.4f}")

    except FileNotFoundError:
        print(f"\nCould not find ground truth file at '{GROUND_TRUTH_PATH}'. Skipping scoring.")
        
    print(f"\n\n🎉 SUCCESS! 🎉")
    print("The script has successfully generated predictions and scored them.")
    print("To generate the REAL submission, change TEST_DATA_PATH to 'dataset/test.csv' and remove the scoring section.")

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, hstack
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBRegressor  # Changed import
from sklearn.metrics import r2_score
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os

# --- 1. Re-usable Components ---

def smape(y_true, y_pred):
    """Calculates the Symmetric Mean Absolute Percentage Error (SMAPE)."""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def clean_text(text):
    """A consistent cleaning function for both text sources."""
    if not isinstance(text, str): return ""
    text = re.sub(r'<.*?>|Value:.*|Unit:.*', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w) > 2]
    return ' '.join(lemmatized_words)

# --- 2. Load and Combine All Data Sources ---

def load_and_combine_all_features(data_dir='processed_data'):
    """
    Loads all preprocessed features and combines them.
    Returns the combined feature matrix and the original DataFrame with prices.
    """
    try:
        print("--- Loading All Preprocessed Data ---")
        
        X_text_struct = load_npz(f'{data_dir}/X_processed.npz')
        y_df = pd.read_csv(f'{data_dir}/y_processed.csv').set_index('sample_id')
        
        vlm_df = pd.read_csv(f'{data_dir}/smolvlm_extracted_features.csv').set_index('sample_id')
        vlm_df = vlm_df.reindex(y_df.index).fillna('')
        vlm_df['cleaned_vlm'] = vlm_df['smolvlm_description'].apply(clean_text)
        
        vlm_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2))
        X_vlm = vlm_vectorizer.fit_transform(vlm_df['cleaned_vlm'])
        
        X_images = np.load(f'{data_dir}/X_image_features.npy')
        all_ids_df = pd.read_csv(f'{data_dir}/all_sample_ids.csv').set_index('sample_id')
        image_features_df = pd.DataFrame(X_images, index=all_ids_df.index)
        X_images_aligned = image_features_df.reindex(y_df.index).values
        
        X_final_combined = hstack([X_text_struct, X_vlm, X_images_aligned]).tocsr()

        print(f"✅ Data loaded and combined successfully! Final shape: {X_final_combined.shape}")
        
        # We need the original dataframe with IDs and prices for splitting and scoring
        full_df = pd.read_csv(f'{data_dir}/y_processed.csv')
        print("Vectorizing VLM text...")
        vlm_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2))
        # This line was changed: we fit_transform on the training data
        X_vlm = vlm_vectorizer.fit_transform(vlm_df['cleaned_vlm'])
        
        # --- ADD THIS LINE ---
        joblib.dump(vlm_vectorizer, f'{data_dir}/vlm_vectorizer.joblib')
        print("Saved VLM TF-IDF vectorizer.")
        return X_final_combined, full_df
        
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Please ensure you have run all three preprocessing scripts first.")
        return None, None

# --- 3. Full Train, Predict, and Evaluate Pipeline ---

def run_full_pipeline(X, df):
    """
    Trains on a subset, predicts on a holdout, saves a submission file, and scores it.
    """
    if X is None or df is None: return

    print("\n--- Simulating Competition Workflow ---")
    
    # --- Step 1: Split the data into a training set and a "pretend" test set ---
    # We split based on the dataframe to keep track of IDs
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Get the corresponding rows from the sparse matrix X
    X_train = X[train_df.index]
    y_train = train_df['price_log']
    
    X_test = X[test_df.index]
    # We will use this later to score our predictions
    y_test_ground_truth_log = test_df['price_log'] 

    print(f"Training on {len(train_df)} samples, predicting on {len(test_df)} samples.")

    # --- Step 2: Train the model ONLY on the training set ---
    print("\nTraining an XGBoost Regressor model...")
    xgb_model = XGBRegressor(
        objective='reg:absoluteerror',  # Matches LightGBM's regression_l1 for MAE
        eval_metric='mae', 
        n_estimators=1000,
        learning_rate=0.05, 
        max_depth=6,  # Approximates num_leaves=31
        random_state=42, 
        n_jobs=-1
    )
    xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test_ground_truth_log)], callbacks=[early_stopping_rounds=50, verbose=False])  # Adjusted for XGBoost callbacks
    print("Model training complete.")

    # --- Step 3: Make predictions on the "pretend" test set ---
    print("Making predictions on the holdout test set...")
    predictions_log = xgb_model.predict(X_test)
    
    # Inverse transform to get actual prices
    predictions_actual = np.expm1(predictions_log)
    predictions_actual[predictions_actual < 0] = 0 # Enforce non-negative constraint

    # --- Step 4: Create and save the submission CSV file ---
    submission_df = pd.DataFrame({
        'sample_id': test_df['sample_id'],
        'price': predictions_actual
    })
    
    submission_path = 'sample_submission_xgb.csv'
    submission_df.to_csv(submission_path, index=False)
    print(f"✅ Submission file created at: '{submission_path}'")

    # --- Step 5: Score the submission file against the ground truth ---
    print("\n--- Scoring Submission File ---")
    
    # Load the ground truth from the original file
    ground_truth_df = pd.read_csv('68e8d1d70b66d_student_resource/student_resource/dataset/sample_test_out.csv')
    
    # Merge our predictions with the ground truth
    # This mimics exactly how the competition leaderboard is calculated
    merged_score_df = pd.merge(submission_df, ground_truth_df, on='sample_id', suffixes=('_pred', '_true'))
    
    if len(merged_score_df) != len(test_df):
        print("Warning: The number of predictions does not match the test set size!")

    # Calculate the final SMAPE score
    final_smape_score = smape(merged_score_df['price_true'], merged_score_df['price_pred'])
    
    print("\n--- FINAL SIMULATED SCORE ---")
    print(f"SMAPE against sample_test_out.csv: {final_smape_score:.4f}%")
    
    return xgb_model

# --- Main execution block ---
if __name__ == '__main__':
    # Load all features and the corresponding dataframe with IDs and prices
    X_final, df_final = load_and_combine_all_features()
    
    # Run the full simulation
    final_model = run_full_pipeline(X_final, df_final)
    
    if final_model:
        joblib.dump(final_model, 'final_xgb_model_for_submission.joblib')
        print("\n✅ Final trained model saved as 'final_xgb_model_for_submission.joblib'")
        print("You can use this model to predict on the real 'test.csv' for the competition.")


In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, hstack
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBRegressor  # Changed import for XGBoost
from sklearn.metrics import r2_score
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os

# --- 1. Re-usable Components ---

def smape(y_true, y_pred):
    """Calculates the Symmetric Mean Absolute Percentage Error (SMAPE)."""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def clean_text(text):
    """A consistent cleaning function for both text sources."""
    if not isinstance(text, str): return ""
    text = re.sub(r'<.*?>|Value:.*|Unit:.*', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w) > 2]
    return ' '.join(lemmatized_words)

# --- 2. Load and Combine All Data Sources ---

def load_and_combine_all_features(data_dir='processed_data'):
    """
    Loads all preprocessed features and combines them.
    Returns the combined feature matrix and the original DataFrame with prices.
    """
    try:
        print("--- Loading All Preprocessed Data ---")
        
        X_text_struct = load_npz(f'{data_dir}/X_processed.npz')
        y_df = pd.read_csv(f'{data_dir}/y_processed.csv').set_index('sample_id')
        
        vlm_df = pd.read_csv(f'{data_dir}/smolvlm_extracted_features.csv').set_index('sample_id')
        vlm_df = vlm_df.reindex(y_df.index).fillna('')
        vlm_df['cleaned_vlm'] = vlm_df['smolvlm_description'].apply(clean_text)
        
        vlm_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2))
        X_vlm = vlm_vectorizer.fit_transform(vlm_df['cleaned_vlm'])
        
        X_images = np.load(f'{data_dir}/X_image_features.npy')
        all_ids_df = pd.read_csv(f'{data_dir}/all_sample_ids.csv').set_index('sample_id')
        image_features_df = pd.DataFrame(X_images, index=all_ids_df.index)
        X_images_aligned = image_features_df.reindex(y_df.index).values
        
        X_final_combined = hstack([X_text_struct, X_vlm, X_images_aligned]).tocsr()

        print(f"✅ Data loaded and combined successfully! Final shape: {X_final_combined.shape}")
        
        # We need the original dataframe with IDs and prices for splitting and scoring
        full_df = pd.read_csv(f'{data_dir}/y_processed.csv')
        print("Vectorizing VLM text...")
        vlm_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2))
        # This line was changed: we fit_transform on the training data
        X_vlm = vlm_vectorizer.fit_transform(vlm_df['cleaned_vlm'])
        
        # --- ADD THIS LINE ---
        joblib.dump(vlm_vectorizer, f'{data_dir}/vlm_vectorizer.joblib')
        print("Saved VLM TF-IDF vectorizer.")
        return X_final_combined, full_df
        
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Please ensure you have run all three preprocessing scripts first.")
        return None, None

# --- 3. Full Train, Predict, and Evaluate Pipeline ---

def run_full_pipeline(X, df):
    """
    Trains on a subset, predicts on a holdout, saves a submission file, and scores it.
    """
    if X is None or df is None: return

    print("\n--- Simulating Competition Workflow ---")
    
    # --- Step 1: Split the data into a training set and a "pretend" test set ---
    # We split based on the dataframe to keep track of IDs
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Get the corresponding rows from the sparse matrix X
    X_train = X[train_df.index]
    y_train = train_df['price_log']
    
    X_test = X[test_df.index]
    # We will use this later to score our predictions
    y_test_ground_truth_log = test_df['price_log'] 

    print(f"Training on {len(train_df)} samples, predicting on {len(test_df)} samples.")

    # --- Step 2: Train the model ONLY on the training set ---
    print("\nTraining an XGBoost Regressor model...")
    xgb_model = XGBRegressor(
        objective='reg:absoluteerror',  # Matches LightGBM's regression_l1 for MAE
        eval_metric='mae', 
        n_estimators=1000,
        learning_rate=0.05, 
        max_depth=6,  # Approximates num_leaves=31
        random_state=42, 
        n_jobs=-1
    )
    xgb_model.fit(X_train, y_train, 
                  eval_set=[(X_test, y_test_ground_truth_log)], 
                  early_stopping_rounds=50, 
                  verbose=False)  # Fixed: Direct arguments to fit(), no callbacks list
    print("Model training complete.")

    # --- Step 3: Make predictions on the "pretend" test set ---
    print("Making predictions on the holdout test set...")
    predictions_log = xgb_model.predict(X_test)
    
    # Inverse transform to get actual prices
    predictions_actual = np.expm1(predictions_log)
    predictions_actual[predictions_actual < 0] = 0 # Enforce non-negative constraint

    # --- Step 4: Create and save the submission CSV file ---
    submission_df = pd.DataFrame({
        'sample_id': test_df['sample_id'],
        'price': predictions_actual
    })
    
    submission_path = 'sample_submission.csv'
    submission_df.to_csv(submission_path, index=False)
    print(f"✅ Submission file created at: '{submission_path}'")

    # --- Step 5: Score the submission file against the ground truth ---
    print("\n--- Scoring Submission File ---")
    
    # Load the ground truth from the original file
    ground_truth_df = pd.read_csv('68e8d1d70b66d_student_resource/student_resource/dataset/sample_test_out.csv')
    
    # Merge our predictions with the ground truth
    # This mimics exactly how the competition leaderboard is calculated
    merged_score_df = pd.merge(submission_df, ground_truth_df, on='sample_id', suffixes=('_pred', '_true'))
    
    if len(merged_score_df) != len(test_df):
        print("Warning: The number of predictions does not match the test set size!")

    # Calculate the final SMAPE score
    final_smape_score = smape(merged_score_df['price_true'], merged_score_df['price_pred'])
    
    print("\n--- FINAL SIMULATED SCORE ---")
    print(f"SMAPE against sample_test_out.csv: {final_smape_score:.4f}%")
    
    return xgb_model

# --- Main execution block ---
if __name__ == '__main__':
    # Load all features and the corresponding dataframe with IDs and prices
    X_final, df_final = load_and_combine_all_features()
    
    # Run the full simulation
    final_model = run_full_pipeline(X_final, df_final)
    
    if final_model:
        joblib.dump(final_model, 'final_xgb_model_for_submission.joblib')
        print("\n✅ Final trained model saved as 'final_xgb_model_for_submission.joblib'")
        print("You can use this model to predict on the real 'test.csv' for the competition.")


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, hstack
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb  # Native XGBoost import for train API
from sklearn.metrics import r2_score
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os

# --- 1. Re-usable Components ---

def smape(y_true, y_pred):
    """Calculates the Symmetric Mean Absolute Percentage Error (SMAPE)."""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def clean_text(text):
    """A consistent cleaning function for both text sources."""
    if not isinstance(text, str): return ""
    text = re.sub(r'<.*?>|Value:.*|Unit:.*', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w) > 2]
    return ' '.join(lemmatized_words)

# --- 2. Load and Combine All Data Sources ---

def load_and_combine_all_features(data_dir='processed_data'):
    """
    Loads all preprocessed features and combines them.
    Returns the combined feature matrix and the original DataFrame with prices.
    """
    try:
        print("--- Loading All Preprocessed Data ---")
        
        X_text_struct = load_npz(f'{data_dir}/X_processed.npz')
        y_df = pd.read_csv(f'{data_dir}/y_processed.csv').set_index('sample_id')
        
        vlm_df = pd.read_csv(f'{data_dir}/smolvlm_extracted_features.csv').set_index('sample_id')
        vlm_df = vlm_df.reindex(y_df.index).fillna('')
        vlm_df['cleaned_vlm'] = vlm_df['smolvlm_description'].apply(clean_text)
        
        vlm_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2))
        X_vlm = vlm_vectorizer.fit_transform(vlm_df['cleaned_vlm'])
        
        X_images = np.load(f'{data_dir}/X_image_features.npy')
        all_ids_df = pd.read_csv(f'{data_dir}/all_sample_ids.csv').set_index('sample_id')
        image_features_df = pd.DataFrame(X_images, index=all_ids_df.index)
        X_images_aligned = image_features_df.reindex(y_df.index).values
        
        X_final_combined = hstack([X_text_struct, X_vlm, X_images_aligned]).tocsr()

        print(f"✅ Data loaded and combined successfully! Final shape: {X_final_combined.shape}")
        
        # We need the original dataframe with IDs and prices for splitting and scoring
        full_df = pd.read_csv(f'{data_dir}/y_processed.csv')
        print("Vectorizing VLM text...")
        vlm_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2))
        # This line was changed: we fit_transform on the training data
        X_vlm = vlm_vectorizer.fit_transform(vlm_df['cleaned_vlm'])
        
        # --- ADD THIS LINE ---
        joblib.dump(vlm_vectorizer, f'{data_dir}/vlm_vectorizer.joblib')
        print("Saved VLM TF-IDF vectorizer.")
        return X_final_combined, full_df
        
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Please ensure you have run all three preprocessing scripts first.")
        return None, None

# --- 3. Full Train, Predict, and Evaluate Pipeline ---

def run_full_pipeline(X, df):
    """
    Trains on a subset, predicts on a holdout, saves a submission file, and scores it.
    """
    if X is None or df is None: return

    print("\n--- Simulating Competition Workflow ---")
    
    # --- Step 1: Split the data into a training set and a "pretend" test set ---
    # We split based on the dataframe to keep track of IDs
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Get the corresponding rows from the sparse matrix X
    X_train = X[train_df.index]
    y_train = train_df['price_log']
    
    X_test = X[test_df.index]
    # We will use this later to score our predictions
    y_test_ground_truth_log = test_df['price_log'] 

    print(f"Training on {len(train_df)} samples, predicting on {len(test_df)} samples.")

    # --- Step 2: Train the model ONLY on the training set ---
    print("\nTraining an XGBoost Regressor model...")
    
    # Create DMatrix for native XGBoost API (handles sparse data)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test_ground_truth_log)
    
    # Parameters equivalent to original LightGBM setup
    params = {
        'objective': 'reg:squarederror',  # For regression; use 'reg:linear' if preferred
        'eval_metric': 'mae',  # Matches original metric
        'learning_rate': 0.05,
        'max_depth': 6,  # Approximates num_leaves=31
        'random_state': 42,  # Seed for reproducibility
        'nthread': -1  # Uses all cores
    }
    
    # Train with native xgb.train and early stopping
    evals = [(dtrain, 'train'), (dtest, 'eval')]
    xgb_model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=evals,
        early_stopping_rounds=50,
        verbose_eval=False
    )
    print("Model training complete.")

    # --- Step 3: Make predictions on the "pretend" test set ---
    print("Making predictions on the holdout test set...")
    predictions_log = xgb_model.predict(dtest, ntree_limit=xgb_model.best_ntree_limit)
    
    # Inverse transform to get actual prices
    predictions_actual = np.expm1(predictions_log)
    predictions_actual[predictions_actual < 0] = 0 # Enforce non-negative constraint

    # --- Step 4: Create and save the submission CSV file ---
    submission_df = pd.DataFrame({
        'sample_id': test_df['sample_id'],
        'price': predictions_actual
    })
    
    submission_path = 'sample_submission.csv'
    submission_df.to_csv(submission_path, index=False)
    print(f"✅ Submission file created at: '{submission_path}'")

    # --- Step 5: Score the submission file against the ground truth ---
    print("\n--- Scoring Submission File ---")
    
    # Load the ground truth from the original file
    ground_truth_df = pd.read_csv('68e8d1d70b66d_student_resource/student_resource/dataset/sample_test_out.csv')
    
    # Merge our predictions with the ground truth
    # This mimics exactly how the competition leaderboard is calculated
    merged_score_df = pd.merge(submission_df, ground_truth_df, on='sample_id', suffixes=('_pred', '_true'))
    
    if len(merged_score_df) != len(test_df):
        print("Warning: The number of predictions does not match the test set size!")

    # Calculate the final SMAPE score
    final_smape_score = smape(merged_score_df['price_true'], merged_score_df['price_pred'])
    
    print("\n--- FINAL SIMULATED SCORE ---")
    print(f"SMAPE against sample_test_out.csv: {final_smape_score:.4f}%")
    
    return xgb_model

# --- Main execution block ---
if __name__ == '__main__':
    # Load all features and the corresponding dataframe with IDs and prices
    X_final, df_final = load_and_combine_all_features()
    
    # Run the full simulation
    final_model = run_full_pipeline(X_final, df_final)
    
    if final_model:
        joblib.dump(final_model, 'final_xgb_model_for_submission.joblib')
        print("\n✅ Final trained model saved as 'final_xgb_model_for_submission.joblib'")
        print("You can use this model to predict on the real 'test.csv' for the competition.")


In [ ]:
!pip install catboost

In [ ]:
# File: train_advanced.py (Fixed for High Dimensionality)

import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import KFold
from sklearn.feature_selection import mutual_info_regression, VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import joblib

def smape(y_true, y_pred):
    """Calculate SMAPE metric."""
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

def load_and_combine_all_features():
    """Load all preprocessed features and combine them."""
    try:
        data_dir = 'processed_data'
        
        print("--- Loading All Preprocessed Data ---")
        
        # Try to load main processed features (npz file)
        try:
            npz_data = np.load(f'{data_dir}/X_processed.npz', allow_pickle=True)
            print(f"✅ X_processed.npz keys: {list(npz_data.keys())}")
            
            # Try different possible array names
            if 'X' in npz_data:
                X_processed = npz_data['X']
            elif 'data' in npz_data:
                X_processed = npz_data['data']
            elif 'arr_0' in npz_data:
                X_processed = npz_data['arr_0']
            else:
                X_processed = npz_data[list(npz_data.keys())[0]]
            
            print(f"   Loaded array shape: {X_processed.shape}")
            
            # Check if it's the right shape
            if X_processed.ndim == 1:
                print(f"   ⚠️  X_processed is 1D with {X_processed.shape[0]} elements")
                print(f"   This doesn't divide evenly by 100 samples (10516 ÷ 100 = 105.16)")
                print(f"   Skipping X_processed.npz - using other features only")
                X_processed = None
            elif X_processed.shape[0] != 100:
                print(f"   ⚠️  X_processed has {X_processed.shape[0]} rows, expected 100")
                print(f"   Skipping X_processed.npz - using other features only")
                X_processed = None
        except Exception as e:
            print(f"   ⚠️  Could not load X_processed.npz properly: {e}")
            print(f"   Continuing with other features...")
            X_processed = None
        
        # Load image features
        X_image = np.load(f'{data_dir}/X_image_features.npy')
        print(f"✅ X_image_features.npy loaded: {X_image.shape}")
        
        # Load text features from smolvlm (use the features_only version if available)
        try:
            df_smolvlm = pd.read_csv(f'{data_dir}/smolvlm_extracted_features_only.csv')
            print(f"✅ smolvlm_extracted_features_only.csv loaded: {df_smolvlm.shape}")
        except:
            df_smolvlm = pd.read_csv(f'{data_dir}/smolvlm_extracted_features.csv')
            print(f"✅ smolvlm_extracted_features.csv loaded: {df_smolvlm.shape}")
        
        # Extract only numeric features (excluding id/text columns)
        smolvlm_features = df_smolvlm.select_dtypes(include=[np.number]).values
        print(f"   Numeric smolvlm features: {smolvlm_features.shape}")
        
        # Load OCR text features
        df_ocr = pd.read_csv(f'{data_dir}/ocr_extracted_text.csv')
        print(f"✅ OCR data loaded: {df_ocr.shape}")
        
        # OCR has 200 rows but we need 100
        ocr_features = df_ocr.select_dtypes(include=[np.number]).values
        if ocr_features.shape[0] != X_image.shape[0]:
            print(f"   ⚠️  OCR shape mismatch: {ocr_features.shape[0]} rows vs {X_image.shape[0]} expected")
            ocr_features = ocr_features[:X_image.shape[0], :]
            print(f"   Adjusted OCR features to: {ocr_features.shape}")
        
        # Load target variable
        df_y = pd.read_csv(f'{data_dir}/y_processed.csv')
        print(f"✅ y_processed.csv loaded: {df_y.shape}")
        
        # Build feature array list
        feature_arrays = [X_image]  # Start with image features
        
        # Add X_processed if it loaded correctly
        if X_processed is not None:
            feature_arrays.insert(0, X_processed)
            print(f"   Including X_processed in features")
        
        # Add smolvlm features if they exist
        if smolvlm_features.shape[1] > 0:
            feature_arrays.append(smolvlm_features)
        
        # Add OCR features if they exist and have features
        if ocr_features.shape[1] > 0:
            feature_arrays.append(ocr_features)
        
        # Ensure all arrays have the same number of rows
        n_samples = X_image.shape[0]
        print(f"\n   Combining {len(feature_arrays)} feature sources...")
        for i, arr in enumerate(feature_arrays):
            print(f"   Array {i}: shape {arr.shape}")
            if arr.shape[0] != n_samples:
                raise ValueError(f"Array {i} has {arr.shape[0]} rows, expected {n_samples}")
        
        X = np.hstack(feature_arrays)
        
        # Create dataframe with target
        if 'price_log' in df_y.columns:
            price_col = 'price_log'
        elif 'target' in df_y.columns:
            price_col = 'target'
        else:
            price_col = df_y.columns[0]
        
        df = pd.DataFrame({'price_log': df_y[price_col].values[:n_samples]})
        
        print(f"\n✅ Data loaded and combined successfully!")
        print(f"   Final feature matrix shape: {X.shape}")
        print(f"   Samples: {X.shape[0]}, Features: {X.shape[1]}")
        print(f"   Feature-to-sample ratio: {X.shape[1]/X.shape[0]:.1f}:1")
        
        if X.shape[1] / X.shape[0] > 10:
            print(f"   ⚠️  HIGH DIMENSIONALITY - Feature selection will be critical!")
        
        return X, df
        
    except FileNotFoundError as e:
        print(f"❌ Error loading files: {e}")
        print("Please check that all files exist in the 'processed_data' directory.")
        return None, None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def reduce_features(X, y, n_features=50):
    """
    Reduce feature dimensionality using multiple strategies.
    """
    print(f"\nOriginal features: {X.shape[1]}")
    
    # Step 1: Remove zero/low variance features
    print("Removing low variance features...")
    var_threshold = VarianceThreshold(threshold=0.01)
    X_var = var_threshold.fit_transform(X)
    print(f"After variance filter: {X_var.shape[1]} features")
    
    # Step 2: Select top features by mutual information
    print(f"Selecting top {n_features} features by mutual information...")
    mi_scores = mutual_info_regression(X_var, y, random_state=42, n_jobs=-1)
    top_features_idx = np.argsort(mi_scores)[-n_features:]
    X_selected = X_var[:, top_features_idx]
    print(f"After MI selection: {X_selected.shape[1]} features")
    
    return X_selected, var_threshold, top_features_idx

def run_advanced_training(X, df, n_splits=5, n_features=50):
    """
    Trains multiple models using K-Fold cross-validation with proper feature selection.
    """
    if X is None or df is None: 
        return

    print(f"\n--- Starting {n_splits}-Fold Cross-Validation Training ---")
    print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")
    print(f"⚠️  Feature-to-sample ratio: {X.shape[1]/X.shape[0]:.1f}:1 (VERY HIGH!)")
    
    y = df['price_log'].values
    
    # Feature reduction BEFORE cross-validation
    X_reduced, var_selector, mi_indices = reduce_features(X, y, n_features=n_features)
    print(f"\n✅ Using {X_reduced.shape[1]} features for training")
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # --- Simplified Models with Strong Regularization ---
    lgb_params = {
        'objective': 'regression_l1',
        'metric': 'mae',
        'random_state': 42,
        'n_estimators': 500,  # Reduced from 2000
        'learning_rate': 0.05,  # Increased from 0.01
        'num_leaves': 15,  # Reduced from 40
        'max_depth': 4,  # Added depth limit
        'min_child_samples': 10,  # Increased from default
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 1.0,  # Increased regularization
        'reg_lambda': 1.0,
        'min_split_gain': 0.01,
        'n_jobs': -1,
        'verbose': -1
    }
    
    xgb_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'seed': 42,
        'n_estimators': 500,
        'learning_rate': 0.05,
        'max_depth': 4,  # Reduced from 7
        'min_child_weight': 5,  # Increased
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_lambda': 1.0,
        'reg_alpha': 1.0,
        'gamma': 0.1,
        'early_stopping_rounds': 50,
        'n_jobs': -1
    }

    cat_params = {
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'random_seed': 42,
        'iterations': 500,
        'learning_rate': 0.05,
        'depth': 4,  # Reduced from 8
        'l2_leaf_reg': 5,  # Increased regularization
        'verbose': 0,
        'allow_writing_files': False
    }

    model_configs = {
        'lgbm': {'model': lgb.LGBMRegressor(**lgb_params), 'preds': np.zeros(len(df))},
        'xgb': {'model': xgb.XGBRegressor(**xgb_params), 'preds': np.zeros(len(df))},
        'cat': {'model': cb.CatBoostRegressor(**cat_params), 'preds': np.zeros(len(df))}
    }

    # --- Cross-Validation Loop ---
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_reduced, y)):
        print(f"\n--- Fold {fold+1}/{n_splits} ---")
        X_train, X_val = X_reduced[train_idx], X_reduced[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        for name, config in model_configs.items():
            print(f"Training {name}...", end=" ")
            
            if name == 'lgbm':
                config['model'].fit(
                    X_train, y_train, 
                    eval_set=[(X_val, y_val)],
                    callbacks=[lgb.early_stopping(50, verbose=False)]
                )
            
            elif name == 'xgb':
                config['model'].fit(
                    X_train, y_train, 
                    eval_set=[(X_val, y_val)],
                    verbose=False
                )
            
            else:  # CatBoost
                config['model'].fit(
                    X_train, y_train, 
                    eval_set=[(X_val, y_val)],
                    early_stopping_rounds=50, 
                    verbose=False
                )
            
            val_preds = config['model'].predict(X_val)
            config['preds'][val_idx] = val_preds
            
            # Calculate fold SMAPE
            fold_smape = smape(np.expm1(y_val), np.expm1(val_preds))
            print(f"Fold SMAPE: {fold_smape:.2f}%")
    
    # --- Evaluation ---
    print("\n" + "="*60)
    print("--- Cross-Validation Results ---")
    print("="*60)
    y_true_actual = np.expm1(y)

    for name, config in model_configs.items():
        preds_actual = np.expm1(config['preds'])
        oof_smape = smape(y_true_actual, preds_actual)
        print(f"{name.upper():8s} SMAPE: {oof_smape:.4f}%")

    # Ensemble with optimized weights
    ensemble_preds_log = (
        0.4 * model_configs['lgbm']['preds'] + 
        0.3 * model_configs['xgb']['preds'] + 
        0.3 * model_configs['cat']['preds']
    )
    
    ensemble_preds_actual = np.expm1(ensemble_preds_log)
    ensemble_smape = smape(y_true_actual, ensemble_preds_actual)
    print(f"{'ENSEMBLE':8s} SMAPE: {ensemble_smape:.4f}%")
    print("="*60)

    # --- Find best model ---
    smape_scores = {
        'lgbm': smape(y_true_actual, np.expm1(model_configs['lgbm']['preds'])),
        'xgb': smape(y_true_actual, np.expm1(model_configs['xgb']['preds'])),
        'cat': smape(y_true_actual, np.expm1(model_configs['cat']['preds'])),
        'ensemble': ensemble_smape
    }
    
    best_model_name = min(smape_scores, key=smape_scores.get)
    print(f"\n🏆 Best model: {best_model_name.upper()} (SMAPE: {smape_scores[best_model_name]:.4f}%)")
    
    # --- Retrain best model on all data ---
    print(f"\nRetraining {best_model_name.upper()} on all data for final submission...")
    
    if best_model_name == 'lgbm':
        final_model = lgb.LGBMRegressor(**lgb_params)
    elif best_model_name == 'xgb':
        final_params = xgb_params.copy()
        final_params.pop('early_stopping_rounds')
        final_model = xgb.XGBRegressor(**final_params)
    elif best_model_name == 'cat':
        final_model = cb.CatBoostRegressor(**cat_params)
    else:  # ensemble
        final_model = lgb.LGBMRegressor(**lgb_params)
    
    final_model.fit(X_reduced, y)
    
    # Save model and feature selectors
    model_package = {
        'model': final_model,
        'var_selector': var_selector,
        'mi_indices': mi_indices,
        'model_type': best_model_name,
        'n_features': n_features
    }
    
    return model_package

# --- Main Execution ---
if __name__ == '__main__':
    result = load_and_combine_all_features()
    
    if result is not None and result[0] is not None:
        X_final, df_final = result
        
        # Try with 50 features first
        model_package = run_advanced_training(X_final, df_final, n_features=50)
        
        if model_package:
            joblib.dump(model_package, 'final_model_package.joblib')
            print("\n✅ Final model package saved as 'final_model_package.joblib'")
            print(f"   Model type: {model_package['model_type']}")
            print(f"   Features used: {model_package['n_features']}")
            print("\n💡 To make predictions, load the package and apply feature selection first!")
    else:
        print("\n❌ Failed to load data. Please check your file paths in load_and_combine_all_features().")